## Na co bacha

- Některé pohledy nutno filtrovat podle "mluvci_jisty", jinak stránky jako *2002ps_stenprot_001schuz_s001012.htm* dělají ze Zaorálka ženu. Ale ne všechny pohledy, pokud chceme vědět, kdo toho napovídal nejvíc – takové projevy se ženou přes víc stránek.

In [1073]:
import os
import polars as pl

In [1074]:
from src.najdi_tituly import najdi_tituly

In [1237]:
import altair as alt

In [1075]:
pl.Config.set_tbl_rows(150)
pl.Config.set_fmt_str_lengths(150)
pl.Config.set_tbl_width_chars(200)

polars.config.Config

In [1076]:
df = pl.scan_parquet(
    "data/projevy.parquet"
).filter(
    pl.col('datum').dt.year() >= 1993
).filter(
    pl.col("komora_komplet").str.contains('Poslanecká') | pl.col('komora_komplet').str.contains('PČR, PS')
).with_columns(
    pl.col('mluvci_id').str.split('/sqw/detail.sqw?id=').alias('id')
).with_columns(
    pl.when(pl.col('id').len() > 2).then(pl.col('id').list.slice(1)).alias('id')
).explode(
    "id"
).with_columns(
    pl.col("text").str.count_matches(" ").alias("pocet_slov") # tady budou zlobit dvojité mezery, pohlídat při čištění
).sort(
    by="datum"
).collect()

In [1241]:
alt.Chart(
    df.group_by_dynamic(index_column="datum",every="1m").agg(pl.col("pocet_slov").sum()).to_pandas()
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("pocet_slov:Q")
)

alt.Chart(...)

In [1245]:
alt.Chart(
    df.group_by_dynamic(index_column="datum",every="1m").agg(pl.col("mluvci").unique().len()).to_pandas()
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("mluvci:Q")
)

alt.Chart(...)

In [ ]:
alt.Chart(
    df.group_by_dynamic(index_column="datum",every="1m").agg(pl.col("pocet_slov").sum()).to_pandas()
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("pocet_slov:Q")
)

In [1077]:
df.filter(pl.col('mluvci_id').is_null()).group_by("obdobi").agg(pl.col('pocet_slov').sum())

obdobi,pocet_slov
i32,u32
1993,993534


In [1078]:
df.filter(pl.col('soubor') == '1996ps_stenprot_016schuz_s016002.htm')

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32
"""Poslanec Miroslav Sládek""","""1996ps_stenprot_016schuz_s016002.htm""","""/sqw/detail.sqw?id=48&o=2""","""Pane předsedající, dámy a pánové, abych nemusel posléze přerušovat řeč, tak bych rád prostřednictvím pana předsedajícího upozornil poslance, že nemusí…",1,"""ps""",1996,16,2,1997-11-11,"""Parlament České republiky, Poslanecká sněmovna 1996 – 1998""","""16. schůze""",true,true,true,"""48&o=2""",2037


In [1079]:
df.group_by('obdobi').agg(pl.col('text').unique().len()).sort(by='obdobi')

obdobi,text
i32,u32
1993,27499
1996,28129
1998,66117
2002,67964
2006,49128
2010,40512
2013,73651
2017,78338
2021,68132


In [1080]:
df.filter(pl.col("id").is_null())

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32
"""Předseda PSP Milan Uhde""","""1993ps_stenprot_001schuz_s001001.htm""",null,"""Vážené paní poslankyně, vážení páni poslanci, vážené dámy a pánové, zahajuji slavnostní schůzi Poslanecké sněmovny Parlamentu České republiky a všechn…",3,"""ps""",1993,1,1,1993-01-01,"""Parlament České republiky, Poslanecká sněmovna 1993 – 1996""","""1. schůze""",true,true,true,null,175
"""Místopředseda PSP Jiří Vlach""","""1993ps_stenprot_001schuz_s001001.htm""",null,"""Vážené dámy a pánové, přistoupíme k prvému bodu pořadu naší schůze, a prosím, aby se slova ujal předseda Poslanecké sněmovny pan Milan Uhde.""",4,"""ps""",1993,1,1,1993-01-01,"""Parlament České republiky, Poslanecká sněmovna 1993 – 1996""","""1. schůze""",true,true,true,null,22
"""Předseda PSP Milan Uhde""","""1993ps_stenprot_001schuz_s001001.htm""",null,"""Vážený pane předsedo vlády, vážení členové vlády, vážení členové diplomatického sboru, vážené paní poslankyně, vážení páni poslanci, vážení představit…",5,"""ps""",1993,1,1,1993-01-01,"""Parlament České republiky, Poslanecká sněmovna 1993 – 1996""","""1. schůze""",true,true,true,null,1075
"""Místopředseda PSP Jiří Vlach""","""1993ps_stenprot_001schuz_s001001.htm""",null,"""Děkuji předsedovi Poslanecké sněmovny panu Milanovi Uhdemu a nyní prosím, aby se ujal slova předseda vlády České republiky pan Václav Klaus.""",6,"""ps""",1993,1,1,1993-01-01,"""Parlament České republiky, Poslanecká sněmovna 1993 – 1996""","""1. schůze""",true,true,true,null,20
"""Předseda vlády ČR Václav Klaus""","""1993ps_stenprot_001schuz_s001001.htm""",null,"""Vážené paní poslankyně, vážení páni poslanci, dámy a pánové, jistě nemusím zdůrazňovat, že jsem si vědom toho, že před vás předstupuji ve chvíli, kter…",7,"""ps""",1993,1,1,1993-01-01,"""Parlament České republiky, Poslanecká sněmovna 1993 – 1996""","""1. schůze""",true,true,true,null,1852
"""Místopředseda PSP Jiří Vlach""","""1993ps_stenprot_001schuz_s001001.htm""",null,"""Děkuji předsedovi vlády panu Václavu Klausovi. Nyní projednáme třetí bod našeho programu, kterým je Prosím předsedu Poslanecké sněmovny pana Milana U…",8,"""ps""",1993,1,1,1993-01-01,"""Parlament České republiky, Poslanecká sněmovna 1993 – 1996""","""1. schůze""",true,true,true,null,26
"""Předseda PSP Milan Uhde""","""1993ps_stenprot_001schuz_s001001.htm""",null,"""Prohlášení České národní rady k parlamentům a národům světa Česká národní rada, jako svobodně zvolený zákonodárný sbor České republiky, pamětliva tisí…",9,"""ps""",1993,1,1,1993-01-01,"""Parlament České republiky, Poslanecká sněmovna 1993 – 1996""","""1. schůze""",true,true,true,null,510
"""Místopředseda PSP Jiří Vlach""","""1993ps_stenprot_001schuz_s001001.htm""",null,"""Děkuji předsedovi Poslanecké sněmovny panu Milanu Uhdemu. Vážené dámy a pánové, dovolte mi, abych vám před závěrem naší slavnostní schůze sdělil ještě…",10,"""ps""",1993,1,1,1993-01-01,"""Parlament České republiky, Poslanecká sněmovna 1993 – 1996""","""1. schůze""",true,true,true,null,124
"""Předseda PSP Milan Uhde""","""1993ps_stenprot_002schuz_s002001.htm""",null,"""Vážené paní poslankyně, vážení páni poslanci, milí hosté, zjišťuji, že podle prezenční listiny bylo v 11.02 hod. podepsáno, a tudíž i přítomno 127 pos…",3,"""ps""",1993,2,1,1993-01-26,"""Parlament České republiky, Poslanecká sněmovna 1993 – 1996""","""2. schůze""",true,true,true,null,559


In [1081]:
df.sample(10)

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32
"""Ministr financí ČR Miroslav Kalousek""","""2010ps_stenprot_040schuz_s040159.htm""","""/sqw/detail.sqw?id=377""","""Děkuji za slovo, paní předsedkyně. Dámy a pánové, k tomu, co zaznělo k programu schůze. Prosím pěkně, mě velmi oslovil návrh pana místopředsedy Zaorál…",10,"""ps""",2010,40,159,2012-06-12,"""Parlament České republiky, Poslanecká sněmovna 2010 – 2013""","""40. schůze""",true,true,true,"""377""",204
"""Místopředsedkyně PSP Petra Buzková""","""1998ps_stenprot_037schuz_s037018.htm""","""/sqw/p.sqw?P=436""","""Děkuji. Vládní jsme v prvém čtení přikázali k projednání čtyřem výborům. Usnesení výborů byla rozdána jako sněmovní tisky 875/1 až 4. Prosím, aby se s…",2,"""ps""",1998,37,18,2001-06-26,"""Parlament České republiky, Poslanecká sněmovna 1998 – 2002""","""37. schůze""",true,true,true,null,39
"""Poslanec Tomáš Helebrant""","""2021ps_stenprot_079schuz_s079157.htm""","""/sqw/detail.sqw?id=6756""","""Děkuji. Vážená paní předsedající, vážené kolegyně, vážení kolegové, pokusit se vyslovit nedůvěru vládě je rozhodně právo každé opozice, ale v situaci,…",5,"""ps""",2021,79,157,2023-10-18,"""PČR, PS 2021-...""","""79. schůze""",true,true,true,"""6756""",374
"""Místopředseda PSP Jan Kasal""","""2002ps_stenprot_022schuz_s022458.htm""","""/sqw/detail.sqw?id=64""","""Myslím si, že je to možné přijmout. Námitky nejsou, takže dám na vás a můžeme se pustit do hlasování.""",4,"""ps""",2002,22,458,2003-11-04,"""Parlament České republiky, Poslanecká sněmovna 2002 – 2006""","""22. schůze""",true,true,true,"""64""",18
"""Místopředsedkyně PSP Klára Dostálová""","""2021ps_stenprot_103schuz_s103055.htm""","""/sqw/detail.sqw?id=6430""","""Děkuji. O všech návrzích bylo tedy hlasováno, přikročíme k hlasování o celém návrhu zákona. Přednesu návrh usnesení: ""Poslanecká sněmovna vyslovuje so…",6,"""ps""",2021,103,55,2024-05-22,"""PČR, PS 2021-...""","""103. schůze""",true,true,true,"""6430""",96
"""Místopředseda PSP Jan Bartošek""","""2013ps_stenprot_042schuz_s042234.htm""","""/sqw/detail.sqw?id=6178""","""Dobře, je to procedurální návrh, o kterém dám hlasovat bez rozpravy. Ještě jednou přivolám kolegy a kolegyně z předsálí. Zahajuji hlasování a táži se,…",10,"""ps""",2013,42,234,2016-03-10,"""Parlament České republiky, Poslanecká sněmovna 2013 – 2017""","""42. schůze""",true,true,true,"""6178""",91
"""Ministr průmyslu a obchodu ČR Vladimír Dlouhý""","""1993ps_stenprot_011schuz_s011017.htm""",null,"""Koneckonců, dámy a pánové, vy jste ti, kteří o tom rozhodují. (Veselost.) Já jsem si to tak ale, pane předsedající, nevysvětlil, takže bych si dovolil…",20,"""ps""",1993,11,17,1993-07-08,"""Parlament České republiky, Poslanecká sněmovna 1993 – 1996""","""11. schůze""",true,true,true,null,797
"""Poslanec David Rath""","""2010ps_stenprot_009schuz_s009145.htm""","""/sqw/detail.sqw?id=5406""","""Pane předsedající, dámy a pánové, já jsem na začátku této poslanecké schůze navrhoval určitý informativní bod premiéra a ministra zdravotnictví o aktu…",10,"""ps""",2010,9,145,2010-12-14,"""Parlament České republiky, Poslanecká sněmovna 2010 – 2013""","""9. schůze""",true,true,true,"""5406""",438
"""Místopředseda PSP Vojtěch Filip""","""2013ps_stenprot_025schuz_s025190.htm""","""/sqw/detail.sqw?id=303""","""Tak, já jsem sice ohlásil pana ministra Brabce, ale mám tady dvě faktické poznámky, a to pana předsedy Stanjury a pana předsedy Kalouska, takže nejdří…",4,"""ps""",2013,25,190,2015-02-11,"""Parlament České republiky, Poslanecká sněmovna 2013 – 2017""","""25. schůze""",true,true,true,"""303""",36


In [1082]:
df_mluvci = pl.read_parquet(os.path.join("data","mluvci.parquet"))

In [1083]:
df_mluvci.filter(pl.col('id').is_null())

id,obdobi,soubor,jmeno,kraj,gender,obdobi_od,obdobi_do,kandidatka,navrhovatel,narozeni_retezec,clenstvi,narozeni_rok,narozeni_datum,jmeno_ciste,tituly
str,i32,str,str,str,str,date,date,str,str,str,list[struct[4]],i64,date,str,list[str]


In [1084]:
df_mluvci.sample(10)

id,obdobi,soubor,jmeno,kraj,gender,obdobi_od,obdobi_do,kandidatka,navrhovatel,narozeni_retezec,clenstvi,narozeni_rok,narozeni_datum,jmeno_ciste,tituly
str,i32,str,str,str,str,date,date,str,str,str,list[struct[4]],i64,date,str,list[str]
"""416""",2006,"""416_5.html""","""JUDr. Cyril Svoboda""","""Praha""","""m""",2006-06-03,2010-06-03,"""KDU-ČSL""",null,"""1956""","[{""člen"",""Výbor pro evropské záležitosti"",""12. 9. 2006"",""9. 1. 2007""}, {""člen"",""Zahraniční výbor"",""12. 9. 2006"",""9. 1. 2007""}, … {null,""Vláda České republiky"",""23. 1. 2009"",""8. 5. 2009""}]",1956,null,"""Cyril Svoboda""","[""JUDr.""]"
"""5292""",2002,"""5292_4.html""","""Ing. Miroslav Váňa""","""Pardubický""","""m""",2002-06-15,2006-06-15,"""ČSSD""",null,"""1961""","[{""člen"",""Výbor pro veřejnou správu, regionální rozvoj a životní prostředí"",""16. 7. 2002"",""15. 6. 2006""}, {""člen"",""Petiční výbor"",""16. 7. 2002"",""15. 6. 2006""}, … {""člen"",""Skupina ČR - Izrael"",""11. 12. 2002"",""15. 6. 2006""}]",1961,null,"""Miroslav Váňa""","[""Ing.""]"
"""346""",2010,"""346_6.html""","""Ing. Petr Bendl""","""Středočeský""","""m""",2010-05-29,2013-08-28,"""ODS""",null,"""1966""","[{""člen"",""Hospodářský výbor"",""7. 7. 2010"",""6. 10. 2011""}, {""člen"",""Kontrolní výbor"",""7. 7. 2010"",""6. 10. 2011""}, … {null,""Vláda České republiky"",""6. 10. 2011"",""10. 7. 2013""}]",1966,null,"""Petr Bendl""","[""Ing.""]"
"""5449""",2006,"""5449_5.html""","""Ing. Jiří Carbol""","""Moravskoslezský""","""m""",2006-06-03,2010-06-03,"""KDU-ČSL""",null,"""1959""","[{""člen"",""Výbor pro zdravotnictví"",""12. 9. 2006"",""3. 6. 2010""}, {""člen"",""Petiční výbor"",""12. 9. 2006"",""3. 6. 2010""}, … {""člen"",""Dozorčí rada Státního fondu dopravní infrastruktury"",""4. 11. 2009"",""23. 3. 2011""}]",1959,null,"""Jiří Carbol""","[""Ing.""]"
"""6154""",2013,"""6154_7.html""","""PhDr. Marek Ženíšek, Ph.D.""","""Plzeňský""","""m""",2013-10-26,2017-10-26,"""TOP09""",null,"""1978""","[{""člen"",""Výbor pro evropské záležitosti"",""4. 12. 2013"",""26. 10. 2017""}, {""člen"",""Výbor pro obranu"",""4. 12. 2013"",""26. 10. 2017""}, … {""člen"",""Správní rada VZP"",""6. 2. 2014"",""24. 1. 2018""}]",1978,null,"""Marek Ženíšek""","[""PhDr."", ""Ph.D.""]"
"""426""",1998,"""426_3.html""","""František Vnouček""","""Středočeský""","""m""",1998-06-20,2002-06-20,"""ČSSD""",null,"""1945""","[{""člen"",""Výbor pro sociální politiku a zdravotnictví"",""22. 7. 1998"",""12. 4. 2000""}, {""člen"",""Hospodářský výbor"",""12. 4. 2000"",""20. 6. 2002""}, … {""člen"",""Skupina ČR - Kanada"",""13. 5. 1999"",""20. 6. 2002""}]",1945,null,"""František Vnouček""",[]
"""5865""",2006,"""5865_5.html""","""Ing. Zdeněk Besta""","""Moravskoslezský""","""m""",2009-07-09,2010-06-03,"""ČSSD""",null,"""1948""","[{""člen"",""Ústavně právní výbor"",""5. 11. 2009"",""3. 6. 2010""}, {""člen"",""Poslanecký klub České strany sociálně demokratické"",""9. 7. 2009"",""3. 6. 2010""}]",1948,null,"""Zdeněk Besta""","[""Ing.""]"
"""5302""",2002,"""5302_4.html""","""RNDr. Marta Bayerová""","""Jihomoravský""","""f""",2002-06-15,2006-06-15,"""KSČM""",null,"""1956""","[{""člen"",""Volební výbor"",""16. 7. 2002"",""15. 6. 2006""}, {""člen"",""Výbor pro veřejnou správu, regionální rozvoj a životní prostředí"",""16. 7. 2002"",""15. 6. 2006""}, … {""člen"",""Skupina ČR - Rakousko"",""6. 3. 2003"",""15. 6. 2006""}]",1956,null,"""Marta Bayerová""","[""RNDr.""]"
"""5257""",2006,"""5257_5.html""","""Ing. Tomáš Dub""","""Praha""","""m""",2006-06-03,2010-06-03,"""ODS""",null,"""1967""","[{""člen"",""Zahraniční výbor"",""12. 9. 2006"",""3. 6. 2010""}, {""místopředsed"",""Zahraniční výbor"",""13. 9. 2006"",""3. 6. 2010""}, … {""člen"",""Skupina ČR - Ázerbájdžán"",""3. 2. 2010"",""3. 6. 2010""}]",1967,null,"""Tomáš Dub""","[""Ing.""]"


In [1085]:
df_mluvci.sort(
    by=['id','jmeno']
).unique(
    subset=['id','jmeno'], keep='first'
)

id,obdobi,soubor,jmeno,kraj,gender,obdobi_od,obdobi_do,kandidatka,navrhovatel,narozeni_retezec,clenstvi,narozeni_rok,narozeni_datum,jmeno_ciste,tituly
str,i32,str,str,str,str,date,date,str,str,str,list[struct[4]],i64,date,str,list[str]
"""5262""",2002,"""5262_4.html""","""MUDr. Vladimír Říha""","""Praha""","""m""",2002-06-15,2006-06-15,"""K""","""KDU-ČSL""","""1949""","[{""člen"",""Ústavně právní výbor"",""16. 7. 2002"",""15. 6. 2006""}, {null,""Ústavně právní výbor"",""17. 7. 2002"",""15. 6. 2006""}, … {""člen"",""Správní rada VZP"",""9. 5. 2006"",""9. 5. 2010""}]",1949,null,"""Vladimír Říha""","[""MUDr.""]"
"""5498""",2006,"""5498_5.html""","""Mgr. Daniel Rovan""","""Praha""","""m""",2006-06-03,2010-06-03,"""ODS""",null,"""1964""","[{""člen"",""Rozpočtový výbor"",""12. 9. 2006"",""3. 6. 2010""}, {null,""Rozpočtový výbor"",""13. 9. 2006"",""3. 6. 2010""}, … {""člen"",""Skupina ČR - Srí Lanka"",""1. 3. 2007"",""3. 6. 2010""}]",1964,null,"""Daniel Rovan""","[""Mgr.""]"
"""6210""",2013,"""6210_7.html""","""Ing. arch. Jaroslav Klaška""","""Jihomoravský""","""m""",2013-10-26,2017-10-26,"""KDU-ČSL""",null,"""1962""","[{""člen"",""Rozpočtový výbor"",""4. 12. 2013"",""26. 10. 2017""}, {""člen"",""Výbor pro evropské záležitosti"",""4. 12. 2013"",""12. 12. 2013""}, … {""předsed"",""Skupina ČR - Chorvatská republika"",""12. 2. 2014"",""26. 10. 2017""}]",1962,null,"""Jaroslav Klaška""","[""Ing."", ""arch.""]"
"""6487""",2021,"""6487_9.html""","""Ing. Patrik Nacher""","""Praha""","""m""",2021-10-09,null,"""ANO2011""",null,"""17. 10. 1974""","[{""člen"",""Organizační výbor"",""23. 11. 2021"",""18. 2. 2022""}, {""člen"",""Hospodářský výbor"",""23. 11. 2021"",null}, … {""člen"",""Skupina ČR - Velká Británie"",""2. 3. 2022"",null}]",1974,1974-10-17,"""Patrik Nacher""","[""Ing.""]"
"""5987""",2013,"""5987_7.html""","""Mgr. Jiří Zemánek""","""Olomoucký""","""m""",2013-10-26,2017-10-26,"""ČSSD""",null,"""1965""","[{""člen"",""Zemědělský výbor"",""4. 12. 2013"",""26. 10. 2017""}, {""člen"",""Rozpočtový výbor"",""4. 12. 2013"",""26. 10. 2017""}, … {""předsed"",""Skupina ČR - Jihoafrická republika"",""13. 2. 2014"",""26. 10. 2017""}]",1965,null,"""Jiří Zemánek""","[""Mgr.""]"
"""6164""",2013,"""6164_7.html""","""Ing. Josef Vondrášek""","""Středočeský""","""m""",2013-10-26,2017-10-26,"""KSČM""",null,"""1950""","[{""člen"",""Rozpočtový výbor"",""4. 12. 2013"",""26. 10. 2017""}, {""člen"",""Podvýbor pro přípravu návrhů na propůjčení nebo udělení státních vyznamenání"",""19. 12. 2013"",""26. 10. 2017""}, … {""předsed"",""Skupina ČR - Indie, Srí Lanka"",""13. 2. 2014"",""26. 10. 2017""}]",1950,null,"""Josef Vondrášek""","[""Ing.""]"
"""5484""",2006,"""5484_5.html""","""Jan Morava""","""Středočeský""","""m""",2006-06-03,2008-09-17,"""ODS""",null,"""1979""","[{""člen"",""Ústavně právní výbor"",""12. 9. 2006"",""17. 9. 2008""}, {""člen"",""Výbor pro vědu, vzdělání, kulturu, mládež a tělovýchovu"",""12. 9. 2006"",""17. 9. 2008""}, … {""předsed"",""Skupina ČR - Německo"",""11. 3. 2008"",""17. 9. 2008""}]",1979,null,"""Jan Morava""",[]
"""6803""",2021,"""6803_9.html""","""PhDr. Jiří Horák, Ph.D.""","""Jihomoravský""","""m""",2021-10-09,null,"""SPOLU""","""KDU-ČSL""","""24. 7. 1984""","[{""člen"",""Zahraniční výbor"",""23. 11. 2021"",null}, {""člen"",""Výbor pro obranu"",""23. 11. 2021"",null}, … {""člen"",""Skupina ČR - Lotyšsko"",""2. 3. 2022"",null}]",1984,1984-07-24,"""Jiří Horák""","[""PhDr."", ""Ph.D.""]"
"""5909""",2021,"""5909_9.html""","""Mgr. Jan Farský""","""Liberecký""","""m""",2021-10-09,2022-02-15,"""PirSTAN""","""STAN""","""11. 7. 1979""","[{""člen"",""Organizační výbor"",""23. 11. 2021"",""25. 1. 2022""}, {""člen"",""Podvýbor pro problematiku exekucí a insolvencí"",""19. 1. 2022"",""15. 2. 2022""}, … {""předsed"",""Poslanecký klub Starostové a nezávislí"",""15. 10. 2021"",""11. 1. 2022""}]",1979,1979-07-11,"""Jan Farský""","[""Mgr.""]"


In [1086]:
df_mluvci.filter(pl.col("jmeno").str.contains("Andrej Babiš"))

id,obdobi,soubor,jmeno,kraj,gender,obdobi_od,obdobi_do,kandidatka,navrhovatel,narozeni_retezec,clenstvi,narozeni_rok,narozeni_datum,jmeno_ciste,tituly
str,i32,str,str,str,str,date,date,str,str,str,list[struct[4]],i64,date,str,list[str]
"""6150""",2013,"""6150_7.html""","""Ing. Andrej Babiš""","""Praha""","""m""",2013-10-26,2017-10-26,"""ANO2011""",null,"""1954""","[{""člen"",""Rozpočtový výbor"",""4. 12. 2013"",""29. 1. 2014""}, {""člen"",""Kontrolní výbor"",""7. 6. 2017"",""26. 10. 2017""}, … {null,""Vláda České republiky"",""29. 1. 2014"",""24. 5. 2017""}]",1954,null,"""Andrej Babiš""","[""Ing.""]"
"""6150""",2017,"""6150_8.html""","""Ing. Andrej Babiš""","""Středočeský""","""m""",2017-10-21,2021-10-20,"""ANO2011""",null,"""1954""","[{""člen"",""Poslanecký klub ANO 2011"",""24. 10. 2017"",""20. 10. 2021""}, {""předsed"",""Vláda České republiky"",""6. 12. 2017"",""27. 6. 2018""}, {""předsed"",""Vláda České republiky"",""27. 6. 2018"",""17. 12. 2021""}]",1954,null,"""Andrej Babiš""","[""Ing.""]"
"""6150""",2021,"""6150_9.html""","""Ing. Andrej Babiš""","""Ústecký""","""m""",2021-10-09,null,"""ANO2011""",null,"""2. 9. 1954""","[{""člen"",""Poslanecký klub ANO 2011"",""12. 10. 2021"",null}, {""předsed"",""Vláda České republiky"",""27. 6. 2018"",""17. 12. 2021""}]",1954,1954-09-02,"""Andrej Babiš""","[""Ing.""]"


In [1087]:
df_mluvci.filter(pl.col("jmeno").str.contains("Tomio Okamura"))

id,obdobi,soubor,jmeno,kraj,gender,obdobi_od,obdobi_do,kandidatka,navrhovatel,narozeni_retezec,clenstvi,narozeni_rok,narozeni_datum,jmeno_ciste,tituly
str,i32,str,str,str,str,date,date,str,str,str,list[struct[4]],i64,date,str,list[str]
"""6105""",2013,"""6105_7.html""","""Tomio Okamura""","""Středočeský""","""m""",2013-10-26,2017-10-26,"""Usvit""",null,"""1972""","[{""člen"",""Organizační výbor"",""26. 3. 2014"",""30. 4. 2015""}, {""člen"",""Podvýbor pro Ústavu České republiky a parlamentní procedury"",""18. 3. 2014"",""26. 10. 2017""}, … {""člen"",""Skupina ČR - Japonsko"",""13. 2. 2014"",""26. 10. 2017""}]",1972,null,"""Tomio Okamura""",[]
"""6105""",2017,"""6105_8.html""","""Tomio Okamura""","""Středočeský""","""m""",2017-10-21,2021-10-20,"""SPD""",null,"""1972""","[{""člen"",""Organizační výbor"",""24. 11. 2017"",""20. 10. 2021""}, {""místopředsed"",""Organizační výbor"",""24. 11. 2017"",""20. 10. 2021""}, … {""předsed"",""Skupina ČR - Afghánská islámská republika"",""9. 3. 2020"",""20. 10. 2021""}]",1972,null,"""Tomio Okamura""",[]
"""6105""",2021,"""6105_9.html""","""Tomio Okamura""","""Moravskoslezský""","""m""",2021-10-09,null,"""SPD""",null,"""4. 7. 1972""","[{""člen"",""Petiční výbor"",""23. 11. 2021"",null}, {""předsed"",""Petiční výbor"",""25. 11. 2021"",null}, … {""člen"",""Skupina ČR - Japonsko"",""21. 2. 2022"",null}]",1972,1972-07-04,"""Tomio Okamura""",[]


In [1088]:
df.sort(by='datum').filter(pl.col('mluvci').str.contains('Petr Fiala'))

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32
"""Ministr školství, mládeže a tělovýchovy ČR Petr Fiala""","""2010ps_stenprot_038schuz_s038048.htm""","""http://vlada.cz/cz/clenove-vlady/prof--phdr--petr-fiala--ph-d---ll-m--95166/""","""Vážená paní předsedkyně, dámy a pánové, Mezinárodní úmluva proti dopingu ve sportu patří do kategorie smluv, jejichž přijetí nebo přijetí jejich změn …",6,"""ps""",2010,38,48,2012-05-03,"""Parlament České republiky, Poslanecká sněmovna 2010 – 2013""","""38. schůze""",true,true,true,null,222
"""Ministr školství, mládeže a tělovýchovy ČR Petr Fiala""","""2010ps_stenprot_040schuz_s040146.htm""","""http://vlada.cz/cz/clenove-vlady/prof--phdr--petr-fiala--ph-d---ll-m--95166/""","""Vážená paní předsedající, dámy a pánové, dovolte mi seznámit vás ve stručnosti s návrhem novely zákona o výkonu ústavní výchovy a ochranné výchovy ve …",9,"""ps""",2010,40,146,2012-06-08,"""Parlament České republiky, Poslanecká sněmovna 2010 – 2013""","""40. schůze""",true,true,true,null,475
"""Ministr školství, mládeže a tělovýchovy ČR Petr Fiala""","""2010ps_stenprot_040schuz_s040147.htm""","""http://vlada.cz/cz/clenove-vlady/prof--phdr--petr-fiala--ph-d---ll-m--95166/""","""Ano, paní předsedající, já souhlasím s tímto návrhem na zkrácení lhůty na 48 hodin.""",8,"""ps""",2010,40,147,2012-06-08,"""Parlament České republiky, Poslanecká sněmovna 2010 – 2013""","""40. schůze""",true,true,true,null,13
"""Ministr školství, mládeže a tělovýchovy ČR Petr Fiala""","""2010ps_stenprot_040schuz_s040244.htm""","""http://vlada.cz/cz/clenove-vlady/prof--phdr--petr-fiala--ph-d---ll-m--95166/""","""Vážená paní poslankyně, dámy a pánové, mám tu čest stát před vámi při interpelacích poprvé, tak mi dovolte na úvod jenom krátkou poznámku. Když jsem p…",5,"""ps""",2010,40,244,2012-06-14,"""Parlament České republiky, Poslanecká sněmovna 2010 – 2013""","""40. schůze""",true,true,true,null,531
"""Ministr školství, mládeže a tělovýchovy ČR Petr Fiala""","""2010ps_stenprot_040schuz_s040244.htm""","""http://vlada.cz/cz/clenove-vlady/prof--phdr--petr-fiala--ph-d---ll-m--95166/""","""Ano, děkuji, pane předsedající, nyní odpovím velmi stručně. Jsem si plně vědom problematické situace v oblasti financování regionálního školství. Jsem…",9,"""ps""",2010,40,244,2012-06-14,"""Parlament České republiky, Poslanecká sněmovna 2010 – 2013""","""40. schůze""",true,true,true,null,110
"""Ministr školství, mládeže a tělovýchovy ČR Petr Fiala""","""2010ps_stenprot_040schuz_s040245.htm""","""http://vlada.cz/cz/clenove-vlady/prof--phdr--petr-fiala--ph-d---ll-m--95166/""","""Vážený pane poslanče, dámy a pánové, děkuji za váš dotaz, který nás přivádí k jiné části širokého resortu, který mám tu čest spravovat, totiž k oblast…",6,"""ps""",2010,40,245,2012-06-14,"""Parlament České republiky, Poslanecká sněmovna 2010 – 2013""","""40. schůze""",true,true,true,null,323
"""Ministr školství, mládeže a tělovýchovy ČR Petr Fiala""","""2010ps_stenprot_040schuz_s040246.htm""","""http://vlada.cz/cz/clenove-vlady/prof--phdr--petr-fiala--ph-d---ll-m--95166/""","""To vysvětlení, pane poslanče, které jsem poskytl, v žádném případě nezpochybňuje jedinečnost výkonů paní Zejdové ani to její momentální úsilí. Podstat…",5,"""ps""",2010,40,246,2012-06-14,"""Parlament České republiky, Poslanecká sněmovna 2010 – 2013""","""40. schůze""",true,true,true,null,43
"""Ministr školství, mládeže a tělovýchovy ČR Petr Fiala""","""2010ps_stenprot_040schuz_s040248.htm""","""http://vlada.cz/cz/clenove-vlady/prof--phdr--petr-fiala--ph-d---ll-m--95166/""","""Vážený pane předsedající, vážený pane poslanče, dámy a pánové, já jsem samozřejmě o takové pomluvě, jak vy říkáte, pane poslanče, o tom, že jsem přiše…",7,"""ps""",2010,40,248,2012-06-14,"""Parlament České republiky, Poslanecká sněmovna 2010 – 2013""","""40. schůze""",true,true,true,nul

In [1089]:
df.filter(pl.col("id") == "6074")

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32
"""Poslanec Petr Fiala""","""2013ps_stenprot_004schuz_s004071.htm""","""/sqw/detail.sqw?id=6074""","""Dámy a pánové, odpověděl bych na otázku paní poslankyně Jourové. Chápu paní poslankyni Jourovou. Doufám, že chápat mohu i bez prostřednictví paní před…",1,"""ps""",2013,4,71,2013-12-16,"""Parlament České republiky, Poslanecká sněmovna 2013 – 2017""","""4. schůze""",true,true,true,"""6074""",264
"""Poslanec Petr Fiala""","""2013ps_stenprot_004schuz_s004071.htm""","""/sqw/detail.sqw?id=6074""","""Děkuji. Na to jsou jednoznačná čísla, kterými to lze doložit, a tyto prostředky jsou prostě potřeba. (Potlesk části poslanců.)""",3,"""ps""",2013,4,71,2013-12-16,"""Parlament České republiky, Poslanecká sněmovna 2013 – 2017""","""4. schůze""",true,true,true,"""6074""",18
"""Poslanec Petr Fiala""","""2013ps_stenprot_004schuz_s004073.htm""","""/sqw/detail.sqw?id=6074""","""Dámy a pánové. Rád bych prostřednictvím paní předsedající sdělil paní poslankyni Jourové, že pokud zmiňovala nadstandardní spolupráci České republiky …",14,"""ps""",2013,4,73,2013-12-16,"""Parlament České republiky, Poslanecká sněmovna 2013 – 2017""","""4. schůze""",true,true,true,"""6074""",107
"""Poslanec Petr Fiala""","""2013ps_stenprot_005schuz_s005002.htm""","""/sqw/detail.sqw?id=6074""","""Děkuji, pane předsedo. Dámy a pánové, je o mně známo, že jsem vcelku důsledným obhájcem parlamentní demokracie, a tudíž bych měl mít radost, že se sch…",2,"""ps""",2013,5,2,2014-01-21,"""Parlament České republiky, Poslanecká sněmovna 2013 – 2017""","""5. schůze""",true,true,true,"""6074""",484
"""Poslanec Petr Fiala""","""2013ps_stenprot_005schuz_s005004.htm""","""/sqw/detail.sqw?id=6074""","""Vážený pane předsedo, dámy a pánové, pan předseda Filip mě tady pochválil jako politologa, já předpokládám, že to udělal proto, aby hned na začátku os…",5,"""ps""",2013,5,4,2014-01-21,"""Parlament České republiky, Poslanecká sněmovna 2013 – 2017""","""5. schůze""",true,true,true,"""6074""",114
"""Poslanec Petr Fiala""","""2013ps_stenprot_005schuz_s005008.htm""","""/sqw/detail.sqw?id=6074""","""Vážený pane předsedo, dámy a pánové, celá ta debata mě jenom utvrdila v tom, že můj příměr s řidičákem je naprosto přesný. Pan Babiš je možná dobrý ři…",6,"""ps""",2013,5,8,2014-01-21,"""Parlament České republiky, Poslanecká sněmovna 2013 – 2017""","""5. schůze""",true,true,true,"""6074""",145
"""Poslanec Petr Fiala""","""2013ps_stenprot_005schuz_s005031.htm""","""/sqw/detail.sqw?id=6074""","""Vážený pane předsedající, dámy a pánové, rád bych vás upozornil na některé problémy, které se pojí s tímto návrhem. Jak už tady bylo řečeno, poslanci …",1,"""ps""",2013,5,31,2014-01-21,"""Parlament České republiky, Poslanecká sněmovna 2013 – 2017""","""5. schůze""",true,true,true,"""6074""",537
"""Poslanec Petr Fiala""","""2013ps_stenprot_005schuz_s005032.htm""","""/sqw/detail.sqw?id=6074""","""Pane předsedající, tady byly naznačeny nějaké motivy, které musím jednoznačně odmítnout. Ta moje výhrada vychází ze zkušenosti. My zavádíme stále nové…",2,"""ps""",2013,5,32,2014-01-21,"""Parlament České republiky, Poslanecká sněmovna 2013 – 2017""","""5. schůze""",true,true,true,"""6074""",207
"""Poslanec Petr Fiala""","""2013ps_stenprot_005schuz_s005032.htm""","""/sqw/detail.sqw?id=6074""","""Já už bych se nehlásil, ale chci odpovědět vaším prostřednictvím, pane místopředsedo, panu poslanci. Já říkám totéž, co jsem říkal tehdy ve vládě Petr…",10,"""ps""",2013,5,32,2014-01-21,"""Parlament České republiky, Poslanecká sněmovna 2013 – 2017""","""5. schůze""",true,true,true,"""6074""",55


In [1090]:
df_mluvci.filter(pl.col("jmeno").str.contains("Fiala"))

id,obdobi,soubor,jmeno,kraj,gender,obdobi_od,obdobi_do,kandidatka,navrhovatel,narozeni_retezec,clenstvi,narozeni_rok,narozeni_datum,jmeno_ciste,tituly
str,i32,str,str,str,str,date,date,str,str,str,list[struct[4]],i64,date,str,list[str]
"""5458""",2006,"""5458_5.html""","""Ing. Jaroslav Fiala, CSc.""","""Karlovarský""","""m""",2006-06-03,2010-06-03,"""ČSSD""",null,"""1956""","[{""člen"",""Výbor pro obranu"",""12. 9. 2006"",""3. 6. 2010""}, {""člen"",""Kontrolní výbor"",""12. 9. 2006"",""3. 6. 2010""}, … {""člen"",""Poslanecký klub České strany sociálně demokratické"",""5. 6. 2006"",""3. 6. 2010""}]",1956,null,"""Jaroslav Fiala""","[""Ing."", ""CSc.""]"
"""5459""",2006,"""5459_5.html""","""Ing. Radim Fiala""","""Olomoucký""","""m""",2006-06-03,2010-06-03,"""ODS""",null,"""1969""","[{""člen"",""Výbor pro životní prostředí"",""12. 9. 2006"",""3. 6. 2010""}, {""člen"",""Rozpočtový výbor"",""12. 9. 2006"",""3. 6. 2010""}, … {""člen"",""Skupina ČR - Čína"",""21. 3. 2007"",""3. 6. 2010""}]",1969,null,"""Radim Fiala""","[""Ing.""]"
"""5459""",2010,"""5459_6.html""","""Ing. Radim Fiala""","""Olomoucký""","""m""",2010-05-29,2013-08-28,"""ODS""",null,"""1969""","[{""člen"",""Rozpočtový výbor"",""7. 7. 2010"",""18. 12. 2012""}, {""člen"",""Výbor pro životní prostředí"",""7. 7. 2010"",""28. 8. 2013""}, … {""místopředsed"",""Skupina ČR - Ruská federace"",""3. 2. 2011"",""28. 8. 2013""}]",1969,null,"""Radim Fiala""","[""Ing.""]"
"""5459""",2013,"""5459_7.html""","""Ing. Radim Fiala""","""Olomoucký""","""m""",2013-10-26,2017-10-26,"""Usvit""",null,"""1969""","[{""člen"",""Rozpočtový výbor"",""4. 12. 2013"",""30. 4. 2015""}, {""člen"",""Organizační výbor"",""4. 12. 2013"",""26. 3. 2014""}, … {""předsed"",""Skupina ČR - Turecká republika"",""13. 2. 2014"",""26. 10. 2017""}]",1969,null,"""Radim Fiala""","[""Ing.""]"
"""6074""",2013,"""6074_7.html""","""Prof. PhDr. Petr Fiala, Ph.D., LL.M.""","""Jihomoravský""","""m""",2013-10-26,2017-10-26,"""ODS""",null,"""1964""","[{""člen"",""Výbor pro evropské záležitosti"",""4. 12. 2013"",""26. 10. 2017""}, {""místopředsed"",""Výbor pro evropské záležitosti"",""5. 12. 2013"",""26. 10. 2017""}, … {""místopředsed"",""Poslanecký klub Občanské demokratické strany"",""6. 11. 2013"",""9. 12. 2014""}]",1964,null,"""Petr Fiala""","[""Prof."", ""PhDr."", … ""LL.M.""]"
"""5459""",2017,"""5459_8.html""","""Ing. Radim Fiala""","""Olomoucký""","""m""",2017-10-21,2021-10-20,"""SPD""",null,"""1969""","[{""člen"",""Organizační výbor"",""28. 11. 2017"",""20. 10. 2021""}, {""člen"",""Hospodářský výbor"",""28. 11. 2017"",""20. 10. 2021""}, … {""člen"",""Skupina ČR - Brazílie"",""18. 3. 2019"",""20. 10. 2021""}]",1969,null,"""Radim Fiala""","[""Ing.""]"
"""6074""",2017,"""6074_8.html""","""Prof. PhDr. Petr Fiala, Ph.D., LL.M.""","""Jihomoravský""","""m""",2017-10-21,2021-10-20,"""ODS""",null,"""1964""","[{""člen"",""Organizační výbor"",""28. 11. 2017"",""20. 10. 2021""}, {""místopředsed"",""Organizační výbor"",""28. 11. 2017"",""20. 10. 2021""}, {""člen"",""Poslanecký klub Občanské demokratické strany"",""24. 10. 2017"",""20. 10. 2021""}]",1964,null,"""Petr Fiala""","[""Prof."", ""PhDr."", … ""LL.M.""]"
"""5459""",2021,"""5459_9.html""","""Ing. Radim Fiala""","""Olomoucký""","""m""",2021-10-09,null,"""SPD""",null,"""24. 7. 1969""","[{""člen"",""Hospodářský výbor"",""23. 11. 2021"",null}, {""člen"",""Organizační výbor"",""23. 11. 2021"",null}, … {""předsed"",""Poslanecký klub Svoboda a přímá demokracie"",""12. 10. 2021"",null}]",1969,1969-07-24,"""Radim Fiala""","[""Ing.""]"
"""6074""",2021,"""6074_9.html""","""Prof. PhDr. Petr Fiala, Ph.D., LL.M.""","""Jihomoravský""","""m""",2021-10-09,null,"""SPOLU""","""ODS""","""1. 9. 1964""","[{""člen"",""Poslanecký klub Občanské demokratické strany"",""12. 10. 2021"",null}, {""předsed"",""Vláda České republiky"",""28. 11. 2021"",null}]",1964,1964-09-01,"""Petr Fiala""","[""Prof."", ""PhDr."", … ""LL.M.""]"


In [1091]:
df.filter(pl.col("mluvci").str.contains("Petr Fiala"))

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32
"""Ministr školství, mládeže a tělovýchovy ČR Petr Fiala""","""2010ps_stenprot_038schuz_s038048.htm""","""http://vlada.cz/cz/clenove-vlady/prof--phdr--petr-fiala--ph-d---ll-m--95166/""","""Vážená paní předsedkyně, dámy a pánové, Mezinárodní úmluva proti dopingu ve sportu patří do kategorie smluv, jejichž přijetí nebo přijetí jejich změn …",6,"""ps""",2010,38,48,2012-05-03,"""Parlament České republiky, Poslanecká sněmovna 2010 – 2013""","""38. schůze""",true,true,true,null,222
"""Ministr školství, mládeže a tělovýchovy ČR Petr Fiala""","""2010ps_stenprot_040schuz_s040146.htm""","""http://vlada.cz/cz/clenove-vlady/prof--phdr--petr-fiala--ph-d---ll-m--95166/""","""Vážená paní předsedající, dámy a pánové, dovolte mi seznámit vás ve stručnosti s návrhem novely zákona o výkonu ústavní výchovy a ochranné výchovy ve …",9,"""ps""",2010,40,146,2012-06-08,"""Parlament České republiky, Poslanecká sněmovna 2010 – 2013""","""40. schůze""",true,true,true,null,475
"""Ministr školství, mládeže a tělovýchovy ČR Petr Fiala""","""2010ps_stenprot_040schuz_s040147.htm""","""http://vlada.cz/cz/clenove-vlady/prof--phdr--petr-fiala--ph-d---ll-m--95166/""","""Ano, paní předsedající, já souhlasím s tímto návrhem na zkrácení lhůty na 48 hodin.""",8,"""ps""",2010,40,147,2012-06-08,"""Parlament České republiky, Poslanecká sněmovna 2010 – 2013""","""40. schůze""",true,true,true,null,13
"""Ministr školství, mládeže a tělovýchovy ČR Petr Fiala""","""2010ps_stenprot_040schuz_s040244.htm""","""http://vlada.cz/cz/clenove-vlady/prof--phdr--petr-fiala--ph-d---ll-m--95166/""","""Vážená paní poslankyně, dámy a pánové, mám tu čest stát před vámi při interpelacích poprvé, tak mi dovolte na úvod jenom krátkou poznámku. Když jsem p…",5,"""ps""",2010,40,244,2012-06-14,"""Parlament České republiky, Poslanecká sněmovna 2010 – 2013""","""40. schůze""",true,true,true,null,531
"""Ministr školství, mládeže a tělovýchovy ČR Petr Fiala""","""2010ps_stenprot_040schuz_s040244.htm""","""http://vlada.cz/cz/clenove-vlady/prof--phdr--petr-fiala--ph-d---ll-m--95166/""","""Ano, děkuji, pane předsedající, nyní odpovím velmi stručně. Jsem si plně vědom problematické situace v oblasti financování regionálního školství. Jsem…",9,"""ps""",2010,40,244,2012-06-14,"""Parlament České republiky, Poslanecká sněmovna 2010 – 2013""","""40. schůze""",true,true,true,null,110
"""Ministr školství, mládeže a tělovýchovy ČR Petr Fiala""","""2010ps_stenprot_040schuz_s040245.htm""","""http://vlada.cz/cz/clenove-vlady/prof--phdr--petr-fiala--ph-d---ll-m--95166/""","""Vážený pane poslanče, dámy a pánové, děkuji za váš dotaz, který nás přivádí k jiné části širokého resortu, který mám tu čest spravovat, totiž k oblast…",6,"""ps""",2010,40,245,2012-06-14,"""Parlament České republiky, Poslanecká sněmovna 2010 – 2013""","""40. schůze""",true,true,true,null,323
"""Ministr školství, mládeže a tělovýchovy ČR Petr Fiala""","""2010ps_stenprot_040schuz_s040246.htm""","""http://vlada.cz/cz/clenove-vlady/prof--phdr--petr-fiala--ph-d---ll-m--95166/""","""To vysvětlení, pane poslanče, které jsem poskytl, v žádném případě nezpochybňuje jedinečnost výkonů paní Zejdové ani to její momentální úsilí. Podstat…",5,"""ps""",2010,40,246,2012-06-14,"""Parlament České republiky, Poslanecká sněmovna 2010 – 2013""","""40. schůze""",true,true,true,null,43
"""Ministr školství, mládeže a tělovýchovy ČR Petr Fiala""","""2010ps_stenprot_040schuz_s040248.htm""","""http://vlada.cz/cz/clenove-vlady/prof--phdr--petr-fiala--ph-d---ll-m--95166/""","""Vážený pane předsedající, vážený pane poslanče, dámy a pánové, já jsem samozřejmě o takové pomluvě, jak vy říkáte, pane poslanče, o tom, že jsem přiše…",7,"""ps""",2010,40,248,2012-06-14,"""Parlament České republiky, Poslanecká sněmovna 2010 – 2013""","""40. schůze""",true,true,true,nul

## Sloučení

In [1093]:
df = df.join(df_mluvci, on=['id','obdobi'], how='left', nulls_equal=False)

In [1094]:
df.filter(pl.col("mluvci").str.contains("Zaorálek"))

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov,soubor_right,jmeno,kraj,gender,obdobi_od,obdobi_do,kandidatka,navrhovatel,narozeni_retezec,clenstvi,narozeni_rok,narozeni_datum,jmeno_ciste,tituly
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32,str,str,str,str,date,date,str,str,str,list[struct[4]],i64,date,str,list[str]
"""Pan poslanec Zaorálek Lubomír""","""1996ps_stenprot_001schuz_s001001.htm""","""/sqw/hlasy.sqw?G=5978""","""Slibuji.""",189,"""ps""",1996,1,1,1996-06-25,"""Parlament České republiky, Poslanecká sněmovna 1996 – 1998""","""1. schůze""",true,true,true,null,0,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Poslanec Lubomír Zaorálek""","""1996ps_stenprot_005schuz_s005049.htm""","""/sqw/hlasy.sqw?G=5978""","""Vážený pane předsedající, vážená sněmovno, vážení poslanci, dovolte, abych vaši pozornost obrátil zpět k rozpočtu. Já bych tady chtěl vyjádřit, co cít…",6,"""ps""",1996,5,49,1996-10-09,"""Parlament České republiky, Poslanecká sněmovna 1996 – 1998""","""5. schůze""",true,true,true,null,562,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Poslanec Lubomír Zaorálek""","""1996ps_stenprot_005schuz_s005056.htm""","""/sqw/hlasy.sqw?G=5978""","""Vážený pane předsedající, vážení poslanci, chtěl bych k tomuto návrhu smlouvy mezi Českou a Polskou republikou o vzájemné pomoci v celních otázkách po…",12,"""ps""",1996,5,56,1996-10-10,"""Parlament České republiky, Poslanecká sněmovna 1996 – 1998""","""5. schůze""",true,true,true,null,132,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Poslanec Lubomír Zaorálek""","""1996ps_stenprot_005schuz_s005057.htm""","""/sqw/hlasy.sqw?G=5978""","""Vážený pane předsedající, vážení poslanci, já se domnívám, že je těžkým dluhem minulosti, který s sebou neseme, izolacionismus české zahraniční politi…",17,"""ps""",1996,5,57,1996-10-10,"""Parlament České republiky, Poslanecká sněmovna 1996 – 1998""","""5. schůze""",true,true,true,null,259,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Poslanec Lubomír Zaorálek""","""1996ps_stenprot_007schuz_s007011.htm""","""/sqw/hlasy.sqw?G=5978""","""Pane předsedající, vážená vládo, vážení poslanci, dovolte mi, abych se ještě jednou vrátil k tématu, které zde dnes už několikrát kolovalo, a to je ke…",2,"""ps""",1996,7,11,1996-12-11,"""Parlament České republiky, Poslanecká sněmovna 1996 – 1998""","""7. schůze""",true,true,true,null,565,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Poslanec Lubomír Zaorálek""","""1996ps_stenprot_007schuz_s007019.htm""","""/sqw/hlasy.sqw?G=5978""","""Vážený pane předsedající, vážená sněmovno, zahraniční výbor skutečně projednal na své 4. schůzi, která se konala 16. října, tento návrh, jak mu bylo u…",29,"""ps""",1996,7,19,1996-12-12,"""Parlament České republiky, Poslanecká sněmovna 1996 – 1998""","""7. schůze""",true,true,true,null,68,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Poslanec Lubomír Zaorálek""","""1996ps_stenprot_007schuz_s007021.htm""","""/sqw/hlasy.sqw?G=5978""","""Vážený pane předsedající, vážená sněmovno, dovolím si jen upozornit na to, že tu existuje určitý rozdíl mezi tím usnesením, které navrhuje výbor pro z…",31,"""ps""",1996,7,21,1996-12-12,"""Parlament České republiky, Poslanecká sněmovna 1996 – 1998""","""7. schůze""",true,true,true,null,512,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Poslanec Lubomír Zaorálek""","""1996ps_stenprot_008schuz_s008047.htm""","""/sqw/hlasy.sqw?G=5978""","""Vážený pane předsedající, vážení poslanci, vím, že lidská paměť je často jedinou odvetou za zločiny. Vím, že lidská paměť je možná jedinou odvetou za …",2,"""ps""",1996,8,47,1997-02-12,"""Parlament České republiky, Poslanecká sněmovna 1996 – 1998""","""8. schůze""",true,true,true,null,1000,null,null,null,null,null,null,nul

In [1095]:
df_mluvci.filter(pl.col('id') == '312')

id,obdobi,soubor,jmeno,kraj,gender,obdobi_od,obdobi_do,kandidatka,navrhovatel,narozeni_retezec,clenstvi,narozeni_rok,narozeni_datum,jmeno_ciste,tituly
str,i32,str,str,str,str,date,date,str,str,str,list[struct[4]],i64,date,str,list[str]
"""312""",1996,"""312_2.html""","""JUDr. Zuzka Bebarová Rujbrová""","""Jihomoravský""","""f""",1996-06-01,1998-06-19,"""KSČM""",null,"""1951""","[{""člen"",""Ústavně právní výbor"",""2. 7. 1996"",""19. 6. 1998""}, {""člen"",""Podvýbor pro zpravodajské služby"",""13. 3. 1997"",""19. 6. 1998""}, … {""místopředsed"",""Poslanecký klub Komunistické strany Čech a Moravy"",""4. 6. 1996"",""19. 6. 1998""}]",1951,null,"""Zuzka Bebarová Rujbrová""","[""JUDr.""]"
"""312""",1998,"""312_3.html""","""JUDr. Zuzka Bebarová Rujbrová""","""Jihomoravský""","""f""",1998-06-20,2002-06-20,"""KSČM""",null,"""1951""","[{""člen"",""Ústavně právní výbor"",""22. 7. 1998"",""28. 1. 2000""}, {""člen"",""Výbor pro veřejnou správu, regionální rozvoj a životní prostředí"",""28. 1. 2000"",""20. 6. 2002""}, … {""člen"",""Skupina ČR - Kazachstán"",""2. 7. 1999"",""20. 6. 2002""}]",1951,null,"""Zuzka Bebarová Rujbrová""","[""JUDr.""]"
"""312""",2002,"""312_4.html""","""JUDr. Zuzka Bebarová Rujbrová""","""Jihomoravský""","""f""",2002-06-15,2006-06-15,"""KSČM""",null,"""1951""","[{""člen"",""Mandátový a imunitní výbor"",""11. 7. 2002"",""15. 6. 2006""}, {""člen"",""Petiční výbor"",""16. 7. 2002"",""15. 6. 2006""}, … {""člen"",""Dozorčí rada České konsolidační agentury"",""15. 3. 2006"",""9. 5. 2007""}]",1951,null,"""Zuzka Bebarová Rujbrová""","[""JUDr.""]"
"""312""",2006,"""312_5.html""","""JUDr. Zuzka Bebarová Rujbrová""","""Jihomoravský""","""f""",2006-06-03,2010-06-03,"""KSČM""",null,"""1951""","[{""člen"",""Výbor pro bezpečnost"",""12. 9. 2006"",""3. 6. 2010""}, {""člen"",""Petiční výbor"",""12. 9. 2006"",""3. 6. 2010""}, … {""člen"",""Dozorčí rada České konsolidační agentury"",""15. 3. 2006"",""9. 5. 2007""}]",1951,null,"""Zuzka Bebarová Rujbrová""","[""JUDr.""]"
"""312""",2010,"""312_6.html""","""JUDr. Zuzka Bebarová Rujbrová""","""Jihomoravský""","""f""",2010-05-29,2013-08-28,"""KSČM""",null,"""1951""","[{""člen"",""Výbor pro obranu a bezpečnost"",""7. 7. 2010"",""6. 12. 2011""}, {""člen"",""Výbor pro bezpečnost"",""13. 12. 2011"",""28. 8. 2013""}, … {""člen"",""Skupina ČR - Vietnam"",""16. 12. 2010"",""28. 8. 2013""}]",1951,null,"""Zuzka Bebarová Rujbrová""","[""JUDr.""]"
"""312""",2013,"""312_7.html""","""JUDr. Zuzka Bebarová Rujbrová""","""Jihomoravský""","""f""",2013-10-26,2017-10-26,"""KSČM""",null,"""1951""","[{""člen"",""Petiční výbor"",""4. 12. 2013"",""26. 10. 2017""}, {""člen"",""Výbor pro bezpečnost"",""4. 12. 2013"",""26. 10. 2017""}, … {""člen"",""Skupina ČR - Kuba"",""12. 1. 2016"",""26. 10. 2017""}]",1951,null,"""Zuzka Bebarová Rujbrová""","[""JUDr.""]"


In [1096]:
df.filter(pl.col("mluvci").str.contains("Zaorálek")).filter(pl.col('gender') == 'f').select(pl.col(['mluvci','mluvci_id','jmeno','id','gender','soubor'])).unique()

mluvci,mluvci_id,jmeno,id,gender,soubor
str,str,str,str,str,str
"""Předseda PS Lubomír Zaorálek""","""/sqw/detail.sqw?id=312""","""JUDr. Zuzka Bebarová Rujbrová""","""312""","""f""","""2002ps_stenprot_001schuz_s001012.htm"""


In [1097]:
klic = df_mluvci.select(pl.col(["jmeno","id"])).unique()
klic.sample(3)

jmeno,id
str,str
"""Taťána Fischerová""","""5325"""
"""JUDr. Vít Bárta""","""5893"""
"""JUDr. Klára Slámová""","""5888"""


In [1098]:
df.group_by('obdobi').agg(pl.col('pocet_slov').sum()).sort(by='obdobi')

obdobi,pocet_slov
i32,u32
1993,3623355
1996,3686739
1998,6685642
2002,7117500
2006,5592617
2010,4879479
2013,8885171
2017,9914101
2021,9414551


In [1099]:
df.filter(pl.col('obdobi') == 2021).group_by('id').agg(pl.col('pocet_slov').sum()).sort(by='pocet_slov',descending=True).join(klic, how='left',on='id')

id,pocet_slov,jmeno
str,u32,str
null,1290203,null
"""6473""",368882,"""PhDr. Olga Richterová, Ph.D."""
"""6178""",362682,"""Ing. Jan Bartošek"""
"""6621""",350770,"""doc. Ing. Karel Havlíček, Ph.D., MBA"""
"""6214""",323622,"""Ing. Věra Kovářová"""
"""6147""",315028,"""Ing. Markéta Pekarová Adamová"""
"""6418""",304297,"""Ing. et Ing. Jan Skopeček, Ph.D."""
"""6463""",283824,"""Ing. Aleš Juchelka"""
"""6105""",269235,"""Tomio Okamura"""


In [1100]:
df.filter(pl.col("id").is_null()).group_by("mluvci").agg(pl.col("datum").unique().len()).sort(by="datum",descending=True)

mluvci,datum
str,u32
"""Poslanec Vojtěch Filip""",311
"""Poslanec Václav Exner""",291
"""Poslanec Dalibor Matulka""",251
"""Místopředsedkyně PSP Petra Buzková""",247
"""Poslanec Miloslav Výborný""",229
"""Poslanec Jiří Payne""",226
"""Poslanec Karel Vymětal""",220
"""Poslanec Vlastimil Tlustý""",213
"""Místopředseda PSP Ivan Langer""",199


In [1101]:
df.filter(pl.col("mluvci").str.contains("Vojtěch Filip")).group_by("mluvci_id").agg(pl.col("text").unique().len()).sort(by="text",descending=True)

mluvci_id,text
str,u32
"""/sqw/detail.sqw?id=303""",27488
"""/sqw/p.sqw?P=452""",746
"""/sqw/hlasy.sqw?G=10693""",185
"""/sqw/hlasy.sqw?G=5978""",151
"""/sqw/hlasy.sqw?G=8471""",83
"""/sqw/detail.sqw?id=48&o=2""",56
"""/ff/ab/33/01.htm""",37
"""/cgi-bin/win/sqw/hlasy.sqw?G=11995""",31
"""/sqw/historie.sqw?T=1&O=3""",18


In [1103]:
df.sort(by='datum').tail(10)

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov,soubor_right,jmeno,kraj,gender,obdobi_od,obdobi_do,kandidatka,navrhovatel,narozeni_retezec,clenstvi,narozeni_rok,narozeni_datum,jmeno_ciste,tituly
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32,str,str,str,str,date,date,str,str,str,list[struct[4]],i64,date,str,list[str]
"""Místopředseda PSP Jan Bartošek""","""2021ps_stenprot_146schuz_s146061.htm""","""/sqw/detail.sqw?id=6178""","""Pan ministr? (Ministr: Souhlas.) Zahajuji hlasování. Ptám se, kdo je pro? Ať zmáčkne tlačítko a zvedne ruku. Kdo je proti? Hlasování s pořadovým čísle…",3,"""ps""",2021,146,61,2025-06-27,"""PČR, PS 2021-...""","""146. schůze""",true,true,true,"""6178""",57,"""6178_9.html""","""Ing. Jan Bartošek""","""Jihočeský""","""m""",2021-10-09,2021-11-10,"""SPOLU""","""KDU-ČSL""","""10. 11. 1971""","[{""člen"",""Organizační výbor"",""10. 11. 2021"",null}, {""místopředsed"",""Organizační výbor"",""10. 11. 2021"",null}, … {""člen"",""Skupina ČR - Thajské království"",""4. 4. 2025"",null}]",1971,1971-11-10,"""Jan Bartošek""","[""Ing.""]"
"""Místopředseda PSP Jan Bartošek""","""2021ps_stenprot_146schuz_s146061.htm""","""/sqw/detail.sqw?id=6178""","""Pan ministr? (Ministr: Souhlas.) Zahajuji hlasování. Ptám se, kdo je pro? Ať zmáčkne tlačítko a zvedne ruku. Kdo je proti? Je to hlasování s pořadovým…",4,"""ps""",2021,146,61,2025-06-27,"""PČR, PS 2021-...""","""146. schůze""",true,true,true,"""6178""",76,"""6178_9.html""","""Ing. Jan Bartošek""","""Jihočeský""","""m""",2021-10-09,2021-11-10,"""SPOLU""","""KDU-ČSL""","""10. 11. 1971""","[{""člen"",""Organizační výbor"",""10. 11. 2021"",null}, {""místopředsed"",""Organizační výbor"",""10. 11. 2021"",null}, … {""člen"",""Skupina ČR - Thajské království"",""4. 4. 2025"",null}]",1971,1971-11-10,"""Jan Bartošek""","[""Ing.""]"
"""Místopředseda PSP Jan Bartošek""","""2021ps_stenprot_146schuz_s146061.htm""","""/sqw/detail.sqw?id=6178""","""Stanovisko? (Zpravodaj: Stanovisko výboru - nedoporučující.) Pan ministr? (Ministr: Nesouhlas.) Zahajuji hlasování. Ptám se, kdo je pro? Ať zmáčkne tl…",5,"""ps""",2021,146,61,2025-06-27,"""PČR, PS 2021-...""","""146. schůze""",true,true,true,"""6178""",60,"""6178_9.html""","""Ing. Jan Bartošek""","""Jihočeský""","""m""",2021-10-09,2021-11-10,"""SPOLU""","""KDU-ČSL""","""10. 11. 1971""","[{""člen"",""Organizační výbor"",""10. 11. 2021"",null}, {""místopředsed"",""Organizační výbor"",""10. 11. 2021"",null}, … {""člen"",""Skupina ČR - Thajské království"",""4. 4. 2025"",null}]",1971,1971-11-10,"""Jan Bartošek""","[""Ing.""]"
"""Místopředseda PSP Jan Bartošek""","""2021ps_stenprot_146schuz_s146061.htm""","""/sqw/detail.sqw?id=6178""","""Pan ministr? (Ministr: Neutrální.) Zahajuji hlasování. Ptám se, kdo je pro? Ať zmáčkne tlačítko a zvedne ruku. Kdo je proti? Hlasování s pořadovým čís…",6,"""ps""",2021,146,61,2025-06-27,"""PČR, PS 2021-...""","""146. schůze""",true,true,true,"""6178""",65,"""6178_9.html""","""Ing. Jan Bartošek""","""Jihočeský""","""m""",2021-10-09,2021-11-10,"""SPOLU""","""KDU-ČSL""","""10. 11. 1971""","[{""člen"",""Organizační výbor"",""10. 11. 2021"",null}, {""místopředsed"",""Organizační výbor"",""10. 11. 2021"",null}, … {""člen"",""Skupina ČR - Thajské království"",""4. 4. 2025"",null}]",1971,1971-11-10,"""Jan Bartošek""","[""Ing.""]"
"""Místopředseda PSP Jan Bartošek""","""2021ps_stenprot_146schuz_s146061.htm""","""/sqw/detail.sqw?id=6178""","""Pan ministr? (Ministr: Neutrální.) Zahajuji hlasování. Ptám se, kdo je pro? Ať zmáčkne tlačítko a zvedne ruku. Kdo je proti? Hlasování s pořadovým čís…",7,"""ps""",2021,146,61,2025-06-27,"""PČR, PS 2021-...""","""146. schůze""",true,true,true,"""6178""",57,"""6178_9.html""","""Ing. Jan Bartošek""","""Jihočeský""","""m""",2021-10-09,2021-11-10,"""SPOLU""","""KDU-ČSL""","""10. 11. 1971""","[{""člen"",""Organizační výbor"",""10. 11. 2021"",null}, {""místopředsed"",""

In [1104]:
df.filter(pl.col("id") == "6105").sort(by="datum").group_by_dynamic(index_column="datum",every="1d").agg(pl.col("pocet_slov").sum())

datum,pocet_slov
date,u32
2013-12-10,258
2013-12-12,474
2014-02-18,290
2014-03-19,1938
2014-03-25,3705
2014-04-30,1033
2014-05-15,695
2014-06-10,452
2014-06-11,1833


In [1105]:
alt.Chart(
    df.filter(pl.col("mluvci").str.contains("Andrej Babiš")).sort(by="datum").group_by_dynamic(index_column="datum",every="1d").agg(pl.col("pocet_slov").sum()).to_pandas()
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("pocet_slov:Q")
)

alt.Chart(...)

In [1106]:
alt.Chart(
    df.filter(pl.col("mluvci").str.contains("Tomio Okamura")).sort(by="datum").group_by_dynamic(index_column="datum",every="1d").agg(pl.col("pocet_slov").sum()).to_pandas()
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("pocet_slov:Q")
)

alt.Chart(...)

In [1107]:
alt.Chart(
    df.sort(by="datum").group_by_dynamic(index_column="datum",every="1mo").agg(pl.col("pocet_slov").sum()).to_pandas(),
    width=1000
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("pocet_slov:Q")
)

alt.Chart(...)

In [1108]:
alt.Chart(
    df.filter(pl.col("mluvci").str.contains("Marek Benda")).sort(by="datum").group_by_dynamic(index_column="datum",every="1d").agg(pl.col("pocet_slov").sum()).to_pandas()
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("pocet_slov:Q")
)

alt.Chart(...)

In [1109]:
df.filter(pl.col('obdobi') == 2021).filter(pl.col('id').is_null()).select(pl.col('mluvci')).unique()

mluvci
str
"""Minulý"""
"""Ministryně životního prostředí ČR Anna Hubáčková"""
"""Místopředseda vlády a ministr práce a sociálních věcí ČR Marian Jurečka"""
"""Místopředseda vlády a ministr pro místní rozvoj ČR Ivan Bartoš"""
"""Markéta Pekarová Adamová"""
"""Ministr dopravy ČR Martin Kupka"""
"""Místopředseda vlády a ministr vnitra ČR Vít Rakušan"""
"""Ministr pro evropské záležitosti ČR Mikuláš Bek"""
"""Předseda vlády ČR Andrej Babiš"""


In [1110]:
df.filter(pl.col('obdobi') == 2021).filter(pl.col('id').is_null())

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov,soubor_right,jmeno,kraj,gender,obdobi_od,obdobi_do,kandidatka,navrhovatel,narozeni_retezec,clenstvi,narozeni_rok,narozeni_datum,jmeno_ciste,tituly
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32,str,str,str,str,date,date,str,str,str,list[struct[4]],i64,date,str,list[str]
"""Ministr obrany ČR Lubomír Metnar""","""2021ps_stenprot_001schuz_s001009.htm""","""https://www.vlada.cz/cz/clenove-vlady/lubomir-metnar-167028/""","""Pane předsedo, děkuji. Já jenom technická. Hlasoval jsem pro, ale nepodařilo se mi přihlásit. Tak to jenom na upřesnění.""",3,"""ps""",2021,1,9,2021-11-10,"""PČR, PS 2021-...""","""1. schůze""",true,true,true,null,18,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Místopředseda vlády, ministr průmyslu a obchodu a ministr dopravy ČR Karel Havlíček""","""2021ps_stenprot_001schuz_s001010.htm""","""https://www.vlada.cz/cz/clenove-vlady/karel-havlicek-173244/""","""Vážený pane předsedající, vážené paní poslankyně, vážení páni poslanci. Nebudu v tuto chvíli mluvit k těm právním náležitostem, určitě o tom ještě v p…",5,"""ps""",2021,1,10,2021-11-10,"""PČR, PS 2021-...""","""1. schůze""",true,true,true,null,266,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Místopředsedkyně vlády a ministryně financí ČR Alena Schillerová""","""2021ps_stenprot_001schuz_s001011.htm""","""https://www.vlada.cz/cz/clenove-vlady/alena-schillerova-167019/""","""Děkuji za slovo. Vážený pane předsedající, vážené paní poslankyně, vážení páni poslanci. Především bych vám všem chtěla pogratulovat ke zvolení a záro…",3,"""ps""",2021,1,11,2021-11-10,"""PČR, PS 2021-...""","""1. schůze""",true,true,true,null,541,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Ministr obrany ČR Lubomír Metnar""","""2021ps_stenprot_001schuz_s001012.htm""","""https://www.vlada.cz/cz/clenove-vlady/lubomir-metnar-167028/""","""Děkuji za slovo. Vážený pane předsedající, vážené paní poslankyně, vážení páni poslanci, já se nebudu vyjadřovat k právním otázkám, já se vyjádřím k d…",3,"""ps""",2021,1,12,2021-11-10,"""PČR, PS 2021-...""","""1. schůze""",true,true,true,null,459,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Ministryně pro místní rozvoj ČR Klára Dostálová""","""2021ps_stenprot_001schuz_s001013.htm""","""https://www.vlada.cz/cz/clenove-vlady/klara-dostalova-162037/""","""Dobrý den, vážené a milé kolegyně, vážení a milí kolegové. Scházíme se v novém volebním období. Ve volebních kampaních jsme všichni deklarovali to sam…",3,"""ps""",2021,1,13,2021-11-10,"""PČR, PS 2021-...""","""1. schůze""",true,true,true,null,385,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Ministr životního prostředí ČR Richard Brabec""","""2021ps_stenprot_001schuz_s001013.htm""","""https://www.vlada.cz/cz/clenove-vlady/richard-brabec-167027/""","""Děkuji za slovo, vážený pane předsedající. Milé kolegyně, vážení kolegové, já tady nebudu určitě obstruovat, jako nikdo z mých kolegů to nedělal, a js…",5,"""ps""",2021,1,13,2021-11-10,"""PČR, PS 2021-...""","""1. schůze""",true,true,true,null,554,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Místopředsedkyně vlády a ministryně financí ČR Alena Schillerová""","""2021ps_stenprot_001schuz_s001035.htm""","""https://www.vlada.cz/cz/clenove-vlady/alena-schillerova-167019/""","""Děkuji, pane předsedající. Už velmi krátce. Byli jste tady, ještě když jsem vystupovala já, a já jsem ve svém projevu, kde jsem představovala našeho k…",3,"""ps""",2021,1,35,2021-11-10,"""PČR, PS 2021-...""","""1. schůze""",true,true,true,null,434,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Místopředsedkyně vlády a ministryně financí ČR Alena Schillerová""","""2021ps_stenprot_001schuz_s001036.htm""","""https://www.vlada.cz/cz/clenove

In [1111]:
df.filter(pl.col('obdobi') == 2021).filter(pl.col('id') == '6105')

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov,soubor_right,jmeno,kraj,gender,obdobi_od,obdobi_do,kandidatka,navrhovatel,narozeni_retezec,clenstvi,narozeni_rok,narozeni_datum,jmeno_ciste,tituly
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32,str,str,str,str,date,date,str,str,str,list[struct[4]],i64,date,str,list[str]
"""Poslanec Tomio Okamura""","""2021ps_stenprot_002schuz_s002001.htm""","""/sqw/detail.sqw?id=6105""","""Vážené dámy a pánové, navrhuji jako první bod dnešní schůze Sněmovny mimořádný bod programu s názvem Diskuse o nepřijatelných opatřeních souvisejících…",5,"""ps""",2021,2,1,2021-11-23,"""PČR, PS 2021-...""","""2. schůze""",true,true,true,"""6105""",129,"""6105_9.html""","""Tomio Okamura""","""Moravskoslezský""","""m""",2021-10-09,null,"""SPD""",null,"""4. 7. 1972""","[{""člen"",""Petiční výbor"",""23. 11. 2021"",null}, {""předsed"",""Petiční výbor"",""25. 11. 2021"",null}, … {""člen"",""Skupina ČR - Japonsko"",""21. 2. 2022"",null}]",1972,1972-07-04,"""Tomio Okamura""",[]
"""Poslanec Tomio Okamura""","""2021ps_stenprot_002schuz_s002004.htm""","""/sqw/detail.sqw?id=6105""","""Ne, ne, já jsem říkal, že nejprve bych to navrhl - já jsem dal dvě varianty... (Pekarová: Ano, prosím.) ...abych dal prostor k tomu pozitivně přistoup…",3,"""ps""",2021,2,4,2021-11-23,"""PČR, PS 2021-...""","""2. schůze""",true,true,true,"""6105""",54,"""6105_9.html""","""Tomio Okamura""","""Moravskoslezský""","""m""",2021-10-09,null,"""SPD""",null,"""4. 7. 1972""","[{""člen"",""Petiční výbor"",""23. 11. 2021"",null}, {""předsed"",""Petiční výbor"",""25. 11. 2021"",null}, … {""člen"",""Skupina ČR - Japonsko"",""21. 2. 2022"",null}]",1972,1972-07-04,"""Tomio Okamura""",[]
"""Poslanec Tomio Okamura""","""2021ps_stenprot_002schuz_s002004.htm""","""/sqw/detail.sqw?id=6105""","""Okamura: Dnešní znamená současné schůze.""",5,"""ps""",2021,2,4,2021-11-23,"""PČR, PS 2021-...""","""2. schůze""",true,true,true,"""6105""",4,"""6105_9.html""","""Tomio Okamura""","""Moravskoslezský""","""m""",2021-10-09,null,"""SPD""",null,"""4. 7. 1972""","[{""člen"",""Petiční výbor"",""23. 11. 2021"",null}, {""předsed"",""Petiční výbor"",""25. 11. 2021"",null}, … {""člen"",""Skupina ČR - Japonsko"",""21. 2. 2022"",null}]",1972,1972-07-04,"""Tomio Okamura""",[]
"""Poslanec Tomio Okamura""","""2021ps_stenprot_002schuz_s002023.htm""","""/sqw/detail.sqw?id=6105""","""Vážené dámy a pánové, jménem SPD navrhuji mimořádný bod na dnešní program schůze Poslanecké sněmovny s názvem Zrušení nouzového stavu. Hnutí SPD nepod…",3,"""ps""",2021,2,23,2021-11-26,"""PČR, PS 2021-...""","""2. schůze""",true,true,true,"""6105""",285,"""6105_9.html""","""Tomio Okamura""","""Moravskoslezský""","""m""",2021-10-09,null,"""SPD""",null,"""4. 7. 1972""","[{""člen"",""Petiční výbor"",""23. 11. 2021"",null}, {""předsed"",""Petiční výbor"",""25. 11. 2021"",null}, … {""člen"",""Skupina ČR - Japonsko"",""21. 2. 2022"",null}]",1972,1972-07-04,"""Tomio Okamura""",[]
"""Poslanec Tomio Okamura""","""2021ps_stenprot_002schuz_s002023.htm""","""/sqw/detail.sqw?id=6105""","""Ano, je to návrh našeho poslaneckého klubu. Děkuji.""",5,"""ps""",2021,2,23,2021-11-26,"""PČR, PS 2021-...""","""2. schůze""",true,true,true,"""6105""",7,"""6105_9.html""","""Tomio Okamura""","""Moravskoslezský""","""m""",2021-10-09,null,"""SPD""",null,"""4. 7. 1972""","[{""člen"",""Petiční výbor"",""23. 11. 2021"",null}, {""předsed"",""Petiční výbor"",""25. 11. 2021"",null}, … {""člen"",""Skupina ČR - Japonsko"",""21. 2. 2022"",null}]",1972,1972-07-04,"""Tomio Okamura""",[]
"""Poslanec Tomio Okamura""","""2021ps_stenprot_002schuz_s002023.htm""","""/sqw/detail.sqw?id=6105""","""Jak jsem sdělil, navrhuji zařadit mnou navrhovaný bod s názvem Zrušení nouzového stavu jako druhý bod po bodu 18, což je návrh na potvrzení předsedů v…",7,"""ps""",2021,2,23,2021-11-26,"""PČR, PS 2021-...""","""2. schůze""",true,true,true,"""

In [1112]:
df.sample(10)

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov,soubor_right,jmeno,kraj,gender,obdobi_od,obdobi_do,kandidatka,navrhovatel,narozeni_retezec,clenstvi,narozeni_rok,narozeni_datum,jmeno_ciste,tituly
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32,str,str,str,str,date,date,str,str,str,list[struct[4]],i64,date,str,list[str]
"""Poslanec Ondřej Babka""","""2017ps_stenprot_104schuz_s104157.htm""","""/sqw/detail.sqw?id=6623""","""Já se omlouvám, já jsem tu proceduru načetl už bez toho stažení. Takže buď bychom hlasovali první o tom stažení, nebo bych musel načíst celou tu proce…",2,"""ps""",2017,104,157,2021-06-02,"""Parlament České republiky, Poslanecká sněmovna 2017 – 2021""","""104. schůze""",true,true,true,"""6623""",26,"""6623_8.html""","""Ing. Ondřej Babka""","""Jihočeský""","""m""",2019-07-02,2021-10-20,"""ANO2011""",null,"""1994""","[{""člen"",""Výbor pro životní prostředí"",""10. 7. 2019"",""20. 10. 2021""}, {""člen"",""Výbor pro veřejnou správu a regionální rozvoj"",""11. 3. 2020"",""10. 3. 2021""}, … {""člen"",""Skupina ČR - Velká Británie"",""10. 7. 2019"",""20. 10. 2021""}]",1994,null,"""Ondřej Babka""","[""Ing.""]"
"""Poslanec Robert Kolář""","""1996ps_stenprot_008schuz_s008094.htm""","""/sqw/hlasy.sqw?G=8471""","""Vážený pane předsedající, vážené kolegyně, vážení kolegové, dovolte, abych vás seznámil s výsledky voleb, které jsme absolvovali v polední přestávce. …",1,"""ps""",1996,8,94,1997-02-26,"""Parlament České republiky, Poslanecká sněmovna 1996 – 1998""","""8. schůze""",true,true,true,null,287,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Předseda PSP Jan Hamáček""","""2013ps_stenprot_007schuz_s007150.htm""","""/sqw/detail.sqw?id=5462""","""Pan předseda Fiala.""",2,"""ps""",2013,7,150,2014-03-25,"""Parlament České republiky, Poslanecká sněmovna 2013 – 2017""","""7. schůze""",true,true,true,"""5462""",2,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""Místopředseda PSP Jan Kasal""","""2006ps_stenprot_048schuz_s048233.htm""","""/sqw/detail.sqw?id=64""","""Děkuji vám, pane ministře. Další interpelací je interpelace pana poslance Břetislava Petra, opět na pana ministra Martina Bursíka.""",8,"""ps""",2006,48,233,2009-02-12,"""Parlament České republiky, Poslanecká sněmovna 2006 – 2010""","""48. schůze""",true,true,true,"""64""",17,"""64_5.html""","""Ing. Jan Kasal""","""Vysočina""","""m""",2006-06-03,2010-06-03,"""KDU-ČSL""",null,"""1951""","[{""člen"",""Organizační výbor"",""12. 9. 2006"",""3. 6. 2010""}, {""místopředsed"",""Organizační výbor"",""12. 9. 2006"",""3. 6. 2010""}, … {""člen"",""Skupina ČR - Bosna a Hercegovina"",""26. 3. 2009"",""3. 6. 2010""}]",1951,null,"""Jan Kasal""","[""Ing.""]"
"""Poslanec Karel Krejza""","""2017ps_stenprot_019schuz_s019139.htm""","""/sqw/detail.sqw?id=6510""","""Děkuji vám, vážený pane předsedající. Dámy a pánové, já si nejdřív dovolím zareagovat na své předřečníky. Například u paní ministryně mě zaujala věta:…",15,"""ps""",2017,19,139,2018-09-14,"""Parlament České republiky, Poslanecká sněmovna 2017 – 2021""","""19. schůze""",true,true,true,"""6510""",150,"""6510_8.html""","""Mgr. Karel Krejza""","""Ústecký""","""m""",2017-10-21,2021-10-20,"""ODS""",null,"""1968""","[{""člen"",""Výbor pro obranu"",""28. 11. 2017"",""20. 10. 2021""}, {""člen"",""Zahraniční výbor"",""28. 11. 2017"",""17. 4. 2019""}, … {""člen"",""Skupina ČR - Severní Makedonie"",""25. 10. 2019"",""20. 10. 2021""}]",1968,null,"""Karel Krejza""","[""Mgr.""]"
"""Místopředseda PSP Vojtěch Filip""","""2013ps_stenprot_026schuz_s026136.htm""","""/sqw/detail.sqw?id=303""","""Děkuji panu předsedovi klubu ODS. Myslím, že jsme rozpravu skončili, a budeme se zabývat tím, co přednesla paní zpravodajka. (Poslanec Stanjura mimo m…",4,"""ps""",2013,26,136,2015-03-31,"""Parlament České republiky, Poslanecká sněmovna 2013 – 2017""","""26. schůze""",true,true,true,"""

In [1113]:
df.select(pl.col("pocet_slov")).sum()

pocet_slov
u32
59799155


"Naše pilotní studie ukazuje, že normální řeč plynula rychlostí 1,75 slov za sekundu (105 slov/min.). Bylo ji však možné zrychlit při rutinních úlohách (mě- síce) až na 3 slova za sekundu (180 slov/min.).
"

In [1115]:
df.select(pl.col("pocet_slov")).sum().item() / (105 * 60 * 8 * 365.25)

3.248435785448106

## Kdo nejvíc mluvil

In [1297]:
df.filter(pl.col("obdobi") == 2021).group_by("mluvci").agg(pl.col("pocet_slov").median()).sort(by="pocet_slov",descending=True)

mluvci,pocet_slov
str,f64
"""Poslankyně Markéta Pekarová Adamová""",1340.0
"""Předseda Národní rozpočtové rady Mojmír Hampl""",1055.0
"""Senátor Stanislav Balík""",815.0
"""Prezident České republiky Miloš Zeman""",725.0
"""Předseda vlády ČR Andrej Babiš""",627.5
"""Senátor Jan Grulich""",611.0
"""Ministr zdravotnictví ČR Adam Vojtěch""",583.0
"""Prezident České republiky Petr Pavel""",572.0
"""Senátor Zdeněk Hraba""",568.0


In [1289]:
df.filter(pl.col("obdobi") == 2021).group_by("mluvci").agg(pl.col("pocet_slov").sum()).sort(by="pocet_slov",descending=True)

mluvci,pocet_slov
str,u32
"""Místopředsedkyně PSP Olga Richterová""",368843
"""Místopředseda PSP Jan Bartošek""",361518
"""Místopředseda PSP Karel Havlíček""",345617
"""Místopředsedkyně PSP Věra Kovářová""",323622
"""Předsedkyně PSP Markéta Pekarová Adamová""",313564
"""Místopředseda PSP Jan Skopeček""",304297
"""Poslanec Tomio Okamura""",269235
"""Poslankyně Alena Schillerová""",263344
"""Poslanec Patrik Nacher""",231267


In [1301]:
soucasne_poslanectvo = df_mluvci.filter(pl.col("obdobi") == 2021).select(pl.col("jmeno_ciste")).to_series().to_list()
soucasne_poslanectvo[0:5]

['Jaroslav Bašta',
 'Petr Bendl',
 'Marek Benda',
 'Pavel Žáček',
 'Antonín Tesařík']

In [1303]:
statistiky_soucasneho_poslanectva = []
for p in soucasne_poslanectvo:
    statistiky_soucasneho_poslanectva.append(
        {
            'jmeno':p,
            'pocet_slov':df.filter((pl.col('obdobi') == 2021) & (pl.col('mluvci').str.contains(p))).select(pl.col('pocet_slov')).sum().item()
        }
    )

In [1307]:
pl.DataFrame(statistiky_soucasneho_poslanectva).sort(by='pocet_slov',descending=True)

jmeno,pocet_slov
str,i64
"""Olga Richterová""",368882
"""Jan Bartošek""",361627
"""Karel Havlíček""",353826
"""Věra Kovářová""",323622
"""Jan Skopeček""",304297
"""Aleš Juchelka""",283824
"""Alena Schillerová""",270995
"""Tomio Okamura""",269235
"""Patrik Nacher""",231267


## Vlády

In [1117]:
import pandas as pd

In [1118]:
vlady = [
    {'vlada': 'první vláda Václava Klause', 'vlada_od': '2. 7. 1992', 'vlada_do': '4. 7. 1996'},
    {'vlada': 'druhá vláda Václava Klause', 'vlada_od': '4. 7. 1996', 'vlada_do': '2. 1. 1998'},
    {'vlada': 'vláda Josefa Tošovského', 'vlada_od': '2. 1. 1998', 'vlada_do': '22. 7. 1998'},
    {'vlada': 'vláda Miloše Zemana', 'vlada_od': '22. 7. 1998', 'vlada_do': '15. 7. 2002'},
    {'vlada': 'vláda Vladimíra Špidly', 'vlada_od': '15. 7. 2002', 'vlada_do': '4. 8. 2004'},
    {'vlada': 'vláda Stanislava Grosse', 'vlada_od': '4. 8. 2004', 'vlada_do': '25. 4. 2005'},
    {'vlada': 'vláda Jiřího Paroubka', 'vlada_od': '25. 4. 2005', 'vlada_do': '4. 9. 2006'},
    {'vlada': 'první vláda Mirka Topolánka', 'vlada_od': '4. 9. 2006', 'vlada_do': '9. 1. 2007'},
    {'vlada': 'druhá vláda Mirka Topolánka', 'vlada_od': '9. 1. 2007', 'vlada_do': '8. 5. 2009'},
    {'vlada': 'vláda Jana Fischera', 'vlada_od': '8. 5. 2009', 'vlada_do': '13. 7. 2010'},
    {'vlada': 'vláda Petra Nečase', 'vlada_od': '13. 7. 2010', 'vlada_do': '10. 7. 2013'},
    {'vlada': 'vláda Jiřího Rusnoka', 'vlada_od': '10. 7. 2013', 'vlada_do': '29. 1. 2014'},
    {'vlada': 'vláda Bohuslava Sobotky', 'vlada_od': '29. 1. 2014', 'vlada_do': '13. 12. 2017', 'koalice_kandidatka':["ANO2011","KDU-ČSL","ČSSD"],'koalice_navrhovatel':[],'opozice_kandidatka':["KSČM","ODS","TOP09","Usvit"], 'opozice_navrhovatel':[]},
    {'vlada': 'první vláda Andreje Babiše', 'vlada_od': '13. 12. 2017', 'vlada_do': '27. 6. 2018', 'koalice_kandidatka':["ANO2011"],'koalice_navrhovatel':[],'opozice_kandidatka':["ČSSD","TOP09","KDU-ČSL","STAN","Piráti","ODS","SPD","KSČM"],  'opozice_navrhovatel':[]},
    {'vlada': 'druhá vláda Andreje Babiše', 'vlada_od': '27. 6. 2018', 'vlada_do': '17. 12. 2021', 'koalice_kandidatka':["ČSSD","ANO2011"],'koalice_navrhovatel':[],'opozice_kandidatka':["TOP09","KDU-ČSL","STAN","Piráti","ODS","SPD","KSČM"], 'opozice_navrhovatel':[]},
    {'vlada': 'vláda Petra Fialy s Piráty', 'vlada_od': '17. 12. 2021', 'vlada_do': '30. 9. 2024', 'koalice_kandidatka':['SPOLU'],'koalice_navrhovatel':['STAN','ODS','KDU-ČSL','TOP09','Piráti'], 'opozice_kandidatka':['ANO2011','SPD'],'opozice_navrhovatel':[]},
    {'vlada': 'vláda Petra Fialy bez Pirátů', 'vlada_od': '1. 10. 2024', 'vlada_do': '30. 9. 2025', 'koalice_kandidatka':['SPOLU'],'koalice_navrhovatel':['STAN','ODS','KDU-ČSL','TOP09'],'opozice_kandidatka':['ANO2011','SPD'],'opozice_navrhovatel':['Piráti']},
]

vlady = pl.DataFrame(
    vlady, infer_schema_length=None
).with_columns(
    pl.col(['vlada_od','vlada_do']).str.to_date()
).with_columns(
    pl.col(['koalice_kandidatka','koalice_navrhovatel','opozice_kandidatka','opozice_navrhovatel']).list.join(separator="*")
)

vlady = vlady.group_by(
    "vlada"
).agg(
    pl.date_range(pl.col("vlada_od"), pl.col("vlada_do"), "1d").alias("datum")
).explode(
    "datum"
).join(
    vlady, how='left', on='vlada'
).filter(
    ~pl.col('koalice_kandidatka').is_null()
)

In [1119]:
vlady.sample(10)

vlada,datum,vlada_od,vlada_do,koalice_kandidatka,koalice_navrhovatel,opozice_kandidatka,opozice_navrhovatel
str,date,date,date,str,str,str,str
"""druhá vláda Andreje Babiše""",2020-02-28,2018-06-27,2021-12-17,"""ČSSD*ANO2011""","""""","""TOP09*KDU-ČSL*STAN*Piráti*ODS*SPD*KSČM""",""""""
"""vláda Bohuslava Sobotky""",2017-12-04,2014-01-29,2017-12-13,"""ANO2011*KDU-ČSL*ČSSD""","""""","""KSČM*ODS*TOP09*Usvit""",""""""
"""vláda Petra Fialy s Piráty""",2024-01-24,2021-12-17,2024-09-30,"""SPOLU""","""STAN*ODS*KDU-ČSL*TOP09*Piráti""","""ANO2011*SPD""",""""""
"""vláda Petra Fialy s Piráty""",2023-02-02,2021-12-17,2024-09-30,"""SPOLU""","""STAN*ODS*KDU-ČSL*TOP09*Piráti""","""ANO2011*SPD""",""""""
"""vláda Bohuslava Sobotky""",2015-09-25,2014-01-29,2017-12-13,"""ANO2011*KDU-ČSL*ČSSD""","""""","""KSČM*ODS*TOP09*Usvit""",""""""
"""vláda Petra Fialy bez Pirátů""",2025-02-07,2024-10-01,2025-09-30,"""SPOLU""","""STAN*ODS*KDU-ČSL*TOP09""","""ANO2011*SPD""","""Piráti"""
"""vláda Bohuslava Sobotky""",2017-02-09,2014-01-29,2017-12-13,"""ANO2011*KDU-ČSL*ČSSD""","""""","""KSČM*ODS*TOP09*Usvit""",""""""
"""vláda Bohuslava Sobotky""",2016-07-03,2014-01-29,2017-12-13,"""ANO2011*KDU-ČSL*ČSSD""","""""","""KSČM*ODS*TOP09*Usvit""",""""""
"""vláda Petra Fialy s Piráty""",2022-12-16,2021-12-17,2024-09-30,"""SPOLU""","""STAN*ODS*KDU-ČSL*TOP09*Piráti""","""ANO2011*SPD""",""""""


In [1120]:
df_vlady = df.filter(
    pl.col("obdobi") >= 2013
).join(vlady, how='left', on='datum').with_columns(
    pl.when(
        pl.col('mluvci_id').str.contains('vlada.cz')
    ).then(
        pl.lit('vláda')
    ).when(
        pl.col('koalice_kandidatka').str.contains(pl.col('kandidatka')) | pl.col('koalice_kandidatka').str.contains(pl.col('navrhovatel'))
    ).then(
        pl.lit('vláda')
    ).when(
        pl.col('koalice_navrhovatel').str.contains(pl.col('navrhovatel')) | pl.col('koalice_navrhovatel').str.contains(pl.col('kandidatka'))
    ).then(
        pl.lit('vláda')
    ).when(
        pl.col('opozice_kandidatka').str.contains(pl.col('kandidatka')) | pl.col('opozice_kandidatka').str.contains(pl.col('navrhovatel'))
    ).then(
        pl.lit('opozice')
    ).when(
        pl.col('opozice_navrhovatel').str.contains(pl.col('navrhovatel')) | pl.col('opozice_navrhovatel').str.contains(pl.col('kandidatka'))
    ).then(
        pl.lit('opozice')
    ).alias('vlada_opo')
).sort(
    by='datum'
)

In [1121]:
df_vlady.group_by(['obdobi','vlada_opo']).agg(pl.col('pocet_slov').sum()).sort(by=['obdobi','vlada_opo']).pivot(index="obdobi",on='vlada_opo')

obdobi,null,opozice,vláda
i32,u32,u32,u32
2013,580622,4590155,3714394
2017,568249,6547849,2832388
2021,377566,5370623,3666362


In [1235]:
df_vlady.group_by(['obdobi','vlada_opo']).agg(pl.col('pocet_slov').sum()).sort(by=['obdobi']).pivot(
    index='obdobi',on='vlada_opo'
).with_columns(
    (pl.col("null") / (pl.col("null") + pl.col("vláda") + pl.col("opozice"))).alias("podil_null")
).with_columns(
    (pl.col("opozice") / (pl.col("null") + pl.col("vláda") + pl.col("opozice"))).alias("podíl_opozice")
)

obdobi,null,opozice,vláda,podil_null,podíl_opozice
i32,u32,u32,u32,f64,f64
2013,580622,4590155,3714394,0.065347,0.516609
2017,568249,6547849,2832388,0.057119,0.658175
2021,377566,5370623,3666362,0.040105,0.57046


In [1223]:
df_vlady.group_by(['obdobi','vlada_opo']).agg(pl.col('id').unique().len()).sort(by=['obdobi']).pivot(
    index='obdobi',on='vlada_opo'
)

obdobi,null,vláda,opozice
i32,u32,u32,u32
2013,157,110,92
2017,71,102,130
2021,20,127,125


In [1123]:
df_vlady.filter((pl.col('obdobi') == 2021) & (pl.col('vlada_opo') == 'opozice')).select(pl.col('mluvci','id')).unique()

mluvci,id
str,str
"""Poslanec Radim Fiala""","""5459"""
"""Poslankyně Ivana Mádlová""","""6732"""
"""Poslankyně Jana Mračková Vildumetzová""","""6471"""
"""Poslanec Jan Volný""","""6166"""
"""Poslanec Lubomír Brož""","""6725"""
"""Poslanec Josef Váňa""","""6946"""
"""Poslankyně Eva Fialová""","""6457"""
"""Poslanec Oldřich Černý""","""6726"""
"""Poslanec Ivan Bartoš""","""6433"""


In [1124]:
df_vlady.filter((pl.col('datum') > pl.date(2022, 1, 1)) & (pl.col('vlada_opo') == 'opozice')).select(pl.col('mluvci','id')).unique()

mluvci,id
str,str
"""Poslankyně Alena Schillerová""","""6545"""
"""Poslankyně Kamila Bláhová""","""6940"""
"""Poslanec Karel Tureček""","""6240"""
"""Poslanec Karel Havlíček""","""6621"""
"""Poslankyně Taťána Malá""","""6468"""
"""Poslankyně Jana Hanzlíková""","""6753"""
"""Poslanec Jiří Kobza""","""6503"""
"""Poslanec Drahoslav Ryba""","""6782"""
"""Místopředseda PSP Aleš Juchelka""","""6463"""


In [1125]:
vyvoj = df_vlady.filter(
    pl.col('vlada_opo') == 'vláda'
).group_by_dynamic(
    index_column='datum',every='1mo'
).agg(
    pl.col('pocet_slov').sum()
).rename(
    {'pocet_slov':'slova_vlada'}
).join(df_vlady.filter(
    pl.col('vlada_opo') == 'opozice'
).group_by_dynamic(
    index_column='datum',every='1mo'
).agg(
    pl.col('pocet_slov').sum()
).rename(
    {'pocet_slov':'slova_opozice'}
), how='left', on='datum'
).with_columns(
    (pl.col('slova_vlada') / pl.col('slova_opozice')).alias('pomer')
)

In [1126]:
alt.Chart(
    vyvoj.to_pandas()
).mark_line(
).encode(
    alt.X('datum:T'),
    alt.Y('pomer:Q')
)

alt.Chart(...)

In [1127]:
def kontrola(jmeno):
    return df_vlady.filter(pl.col('mluvci').str.contains(jmeno)).group_by(['obdobi','vlada_opo']).agg(pl.col('text').unique().len()).sort(by='obdobi')

In [1128]:
kontrola('Tomio')

obdobi,vlada_opo,text
i32,str,u32
2013,"""opozice""",391
2013,null,3
2017,null,52
2017,"""opozice""",7373
2021,"""opozice""",841


In [1129]:
kontrola('Babiš')

obdobi,vlada_opo,text
i32,str,u32
2013,"""vláda""",678
2017,"""vláda""",673
2021,"""opozice""",552
2021,"""vláda""",2


In [1130]:
kontrola('Petr Fiala')

obdobi,vlada_opo,text
i32,str,u32
2013,null,13
2013,"""opozice""",246
2017,"""opozice""",6885
2021,"""vláda""",483
2021,null,1
2021,"""opozice""",1


In [1131]:
kontrola('Kateřina Konečná')

obdobi,vlada_opo,text
i32,str,u32
2013,"""opozice""",53
2013,null,1


In [1132]:
kontrola('Marek Benda')

obdobi,vlada_opo,text
i32,str,u32
2013,"""opozice""",345
2013,null,5
2017,"""opozice""",460
2021,"""vláda""",407
2021,"""opozice""",4


In [1133]:
kontrola('Ivan Bartoš')

obdobi,vlada_opo,text
i32,str,u32
2017,null,2
2017,"""opozice""",323
2021,"""opozice""",58
2021,null,30
2021,"""vláda""",253


In [1134]:
kontrola('Schillerová')

obdobi,vlada_opo,text
i32,str,u32
2017,"""vláda""",744
2021,"""opozice""",1161
2021,"""vláda""",23


In [1135]:
df_vlady.filter(pl.col("mluvci").str.contains("Ivan Bartoš")).filter(pl.col("vlada_opo").is_null())

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov,soubor_right,jmeno,kraj,gender,obdobi_od,obdobi_do,kandidatka,navrhovatel,narozeni_retezec,clenstvi,narozeni_rok,narozeni_datum,jmeno_ciste,tituly,vlada,vlada_od,vlada_do,koalice_kandidatka,koalice_navrhovatel,opozice_kandidatka,opozice_navrhovatel,vlada_opo
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32,str,str,str,str,date,date,str,str,str,list[struct[4]],i64,date,str,list[str],str,date,date,str,str,str,str,str
"""Poslanec Ivan Bartoš""","""2017ps_stenprot_001schuz_s001043.htm""","""/sqw/detail.sqw?id=6433""","""Vážený pane předsedo, vážené dámy, vážení pánové, veřejnosti, teď jsme viděli volbu, která skutečně nenaplnila to, co jsme tady vlastně celou dobu sly…",11,"""ps""",2017,1,43,2017-11-24,"""Parlament České republiky, Poslanecká sněmovna 2017 – 2021""","""1. schůze""",true,true,true,"""6433""",158,"""6433_8.html""","""PhDr. Ivan Bartoš, Ph.D.""","""Středočeský""","""m""",2017-10-21,2021-10-20,"""Piráti""",null,"""1980""","[{""člen"",""Výbor pro veřejnou správu a regionální rozvoj"",""28. 11. 2017"",""20. 10. 2021""}, {""člen"",""Výbor pro bezpečnost"",""28. 11. 2017"",""15. 12. 2017""}, … {""člen"",""Skupina ČR - Jamajka"",""12. 6. 2018"",""20. 10. 2021""}]",1980,null,"""Ivan Bartoš""","[""PhDr."", ""Ph.D.""]","""vláda Bohuslava Sobotky""",2014-01-29,2017-12-13,"""ANO2011*KDU-ČSL*ČSSD""","""""","""KSČM*ODS*TOP09*Usvit""","""""",null
"""Poslanec Ivan Bartoš""","""2017ps_stenprot_002schuz_s002006.htm""","""/sqw/detail.sqw?id=6433""","""Vážený pane předsedo, vážené kolegyně, vážení kolegové, tady už zaznělo od mých předřečníků i od pana Vondráčka, jak jemné nuance můžou být v interpre…",2,"""ps""",2017,2,6,2017-11-28,"""Parlament České republiky, Poslanecká sněmovna 2017 – 2021""","""2. schůze""",true,true,true,"""6433""",238,"""6433_8.html""","""PhDr. Ivan Bartoš, Ph.D.""","""Středočeský""","""m""",2017-10-21,2021-10-20,"""Piráti""",null,"""1980""","[{""člen"",""Výbor pro veřejnou správu a regionální rozvoj"",""28. 11. 2017"",""20. 10. 2021""}, {""člen"",""Výbor pro bezpečnost"",""28. 11. 2017"",""15. 12. 2017""}, … {""člen"",""Skupina ČR - Jamajka"",""12. 6. 2018"",""20. 10. 2021""}]",1980,null,"""Ivan Bartoš""","[""PhDr."", ""Ph.D.""]","""vláda Bohuslava Sobotky""",2014-01-29,2017-12-13,"""ANO2011*KDU-ČSL*ČSSD""","""""","""KSČM*ODS*TOP09*Usvit""","""""",null
"""Poslanec Ivan Bartoš""","""2021ps_stenprot_118schuz_s118002.htm""","""/sqw/detail.sqw?o=9&id=6433""","""Ještě jsem stále předseda Pirátské strany. Děkuji.""",3,"""ps""",2021,118,2,2024-11-06,"""PČR, PS 2021-...""","""118. schůze""",true,true,true,null,6,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""vláda Petra Fialy bez Pirátů""",2024-10-01,2025-09-30,"""SPOLU""","""STAN*ODS*KDU-ČSL*TOP09""","""ANO2011*SPD""","""Piráti""",null
"""Poslanec Ivan Bartoš""","""2021ps_stenprot_118schuz_s118002.htm""","""/sqw/detail.sqw?o=9&id=6433""","""Volby vedení Pirátské strany jsou až v sobotu, kdy bude zvoleno nové vedení. Děkuji.""",5,"""ps""",2021,118,2,2024-11-06,"""PČR, PS 2021-...""","""118. schůze""",true,true,true,null,13,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""vláda Petra Fialy bez Pirátů""",2024-10-01,2025-09-30,"""SPOLU""","""STAN*ODS*KDU-ČSL*TOP09""","""ANO2011*SPD""","""Piráti""",null
"""Poslanec Ivan Bartoš""","""2021ps_stenprot_118schuz_s118002.htm""","""/sqw/detail.sqw?o=9&id=6433""","""Vážená paní předsedající, vážení páni ministři, vážené poslankyně, vážení poslanci, my jsme se dnes sešli kvůli takzvanému bypassu digitalizace staveb…",7,"""ps""",2021,118,2,2024-11-06,"""PČR, PS 2021-...""","""118. schůze""",true,true,true,null,897,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""vláda Petra Fialy bez Pirátů""",2024-10-01,2025-09-30,"""SPOLU""","""STAN*ODS*KDU-ČSL*TOP09""","""ANO2011*SPD""","""Piráti""",null
""

In [1137]:
df_vlady.filter(pl.col('datum').dt.year() == 2021).filter(pl.col('datum').dt.month() == 11).filter(pl.col('mluvci').str.contains('Stanjura'))

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov,soubor_right,jmeno,kraj,gender,obdobi_od,obdobi_do,kandidatka,navrhovatel,narozeni_retezec,clenstvi,narozeni_rok,narozeni_datum,jmeno_ciste,tituly,vlada,vlada_od,vlada_do,koalice_kandidatka,koalice_navrhovatel,opozice_kandidatka,opozice_navrhovatel,vlada_opo
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32,str,str,str,str,date,date,str,str,str,list[struct[4]],i64,date,str,list[str],str,date,date,str,str,str,str,str
"""Poslanec Zbyněk Stanjura""","""2021ps_stenprot_001schuz_s001006.htm""","""/sqw/detail.sqw?id=5969""","""Děkuji za slovo. My jsme se dohodli s ostatními předsedy poslaneckých klubů, že počet členů mandátového a imunitního výboru Poslanecká sněmovna stanov…",3,"""ps""",2021,1,6,2021-11-08,"""PČR, PS 2021-...""","""1. schůze""",true,true,true,"""5969""",27,"""5969_9.html""","""Ing. Zbyněk Stanjura""","""Moravskoslezský""","""m""",2021-10-09,null,"""SPOLU""","""ODS""","""15. 2. 1964""","[{""člen"",""Poslanecký klub Občanské demokratické strany"",""12. 10. 2021"",null}, {""předsed"",""Poslanecký klub Občanské demokratické strany"",""12. 10. 2021"",""13. 12. 2021""}, … {""člen"",""Dozorčí rada Státního fondu dopravní infrastruktury"",""15. 5. 2018"",""6. 4. 2022""}]",1964,1964-02-15,"""Zbyněk Stanjura""","[""Ing.""]","""druhá vláda Andreje Babiše""",2018-06-27,2021-12-17,"""ČSSD*ANO2011""","""""","""TOP09*KDU-ČSL*STAN*Piráti*ODS*SPD*KSČM""","""""","""opozice"""
"""Poslanec Zbyněk Stanjura""","""2021ps_stenprot_001schuz_s001010.htm""","""/sqw/detail.sqw?id=5969""","""Myslím, že je to úplně jednoduché. Pan ministr má přednostní právo. (Předsedající: Má, ale ústní.) Takže může vystoupit jako první a nemusíme se o tom…",3,"""ps""",2021,1,10,2021-11-10,"""PČR, PS 2021-...""","""1. schůze""",true,true,true,"""5969""",41,"""5969_9.html""","""Ing. Zbyněk Stanjura""","""Moravskoslezský""","""m""",2021-10-09,null,"""SPOLU""","""ODS""","""15. 2. 1964""","[{""člen"",""Poslanecký klub Občanské demokratické strany"",""12. 10. 2021"",null}, {""předsed"",""Poslanecký klub Občanské demokratické strany"",""12. 10. 2021"",""13. 12. 2021""}, … {""člen"",""Dozorčí rada Státního fondu dopravní infrastruktury"",""15. 5. 2018"",""6. 4. 2022""}]",1964,1964-02-15,"""Zbyněk Stanjura""","[""Ing.""]","""druhá vláda Andreje Babiše""",2018-06-27,2021-12-17,"""ČSSD*ANO2011""","""""","""TOP09*KDU-ČSL*STAN*Piráti*ODS*SPD*KSČM""","""""","""opozice"""
"""Poslanec Zbyněk Stanjura""","""2021ps_stenprot_001schuz_s001017.htm""","""/sqw/detail.sqw?id=5969""","""Děkuji za slovo. Já k tomu bodu vystoupím někdy později. Jenom krátká poznámka k nové politické kultuře, budu se tomu pak věnovat podrobněji. My jsme …",3,"""ps""",2021,1,17,2021-11-10,"""PČR, PS 2021-...""","""1. schůze""",true,true,true,"""5969""",134,"""5969_9.html""","""Ing. Zbyněk Stanjura""","""Moravskoslezský""","""m""",2021-10-09,null,"""SPOLU""","""ODS""","""15. 2. 1964""","[{""člen"",""Poslanecký klub Občanské demokratické strany"",""12. 10. 2021"",null}, {""předsed"",""Poslanecký klub Občanské demokratické strany"",""12. 10. 2021"",""13. 12. 2021""}, … {""člen"",""Dozorčí rada Státního fondu dopravní infrastruktury"",""15. 5. 2018"",""6. 4. 2022""}]",1964,1964-02-15,"""Zbyněk Stanjura""","[""Ing.""]","""druhá vláda Andreje Babiše""",2018-06-27,2021-12-17,"""ČSSD*ANO2011""","""""","""TOP09*KDU-ČSL*STAN*Piráti*ODS*SPD*KSČM""","""""","""opozice"""
"""Poslanec Zbyněk Stanjura""","""2021ps_stenprot_001schuz_s001029.htm""","""/sqw/detail.sqw?id=5969""","""Děkuji za slovo. Určitě ode mě nemůžete čekat, že bych kritizoval to, že někdo vystupuje, to určitě ne. Věta o tom obědě je má z minulého volebního ob…",6,"""ps""",2021,1,29,2021-11-10,"""PČR, PS 2021-...""","""1. schůze""",true,true,true,"""5969""",300,"""5969_9.html""","""Ing. Zbyněk Stanjura""","""Moravskoslezský""","""m""",2021-10-09,null,"""SPOLU""","""ODS""","""15. 2. 1964"

In [1138]:
df_vlady.filter(pl.col('datum').dt.year() == 2021).filter(pl.col('datum').dt.month() == 11).select(pl.col(['mluvci','vlada_opo','datum'])).unique()

mluvci,vlada_opo,datum
str,str,date
"""Poslanec Milan Brázdil""","""vláda""",2021-11-23
"""Poslanec Pavel Žáček""","""opozice""",2021-11-10
"""Poslanec Tomio Okamura""","""opozice""",2021-11-23
"""Poslanec Radim Fiala""","""opozice""",2021-11-10
"""Poslanec Martin Kolovratník""","""vláda""",2021-11-08
"""Poslanec Stanislav Berkovec""","""vláda""",2021-11-10
"""Ministr životního prostředí ČR Richard Brabec""","""vláda""",2021-11-23
"""Poslankyně Jana Černochová""","""opozice""",2021-11-10
"""Místopředsedkyně PSP Věra Kovářová""","""opozice""",2021-11-23


In [1139]:
df_vlady.filter(pl.col('vlada_opo') == 'vláda').sample(5)

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov,soubor_right,jmeno,kraj,gender,obdobi_od,obdobi_do,kandidatka,navrhovatel,narozeni_retezec,clenstvi,narozeni_rok,narozeni_datum,jmeno_ciste,tituly,vlada,vlada_od,vlada_do,koalice_kandidatka,koalice_navrhovatel,opozice_kandidatka,opozice_navrhovatel,vlada_opo
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32,str,str,str,str,date,date,str,str,str,list[struct[4]],i64,date,str,list[str],str,date,date,str,str,str,str,str
"""Místopředseda PSP Jan Skopeček""","""2021ps_stenprot_008schuz_s008144.htm""","""/sqw/detail.sqw?id=6418""","""Děkuji pěkně. Než dám slovo zpravodaji a poprosím ho o úvodní slovo, tak přečtu omluvy. Dnes od 14 do 19 hodin se z pracovních důvodů omlouvá ministr …",2,"""ps""",2021,8,144,2022-02-02,"""PČR, PS 2021-...""","""8. schůze""",true,true,true,"""6418""",112,"""6418_9.html""","""Ing. et Ing. Jan Skopeček, Ph.D.""","""Středočeský""","""m""",2021-10-09,2021-11-10,"""SPOLU""","""ODS""","""21. 10. 1980""","[{""člen"",""Organizační výbor"",""10. 11. 2021"",null}, {""místopředsed"",""Organizační výbor"",""10. 11. 2021"",null}, … {""místopředsed"",""Poslanecký klub Občanské demokratické strany"",""12. 10. 2021"",null}]",1980,1980-10-21,"""Jan Skopeček""","[""Ing."", ""Ing."", ""Ph.D.""]","""vláda Petra Fialy s Piráty""",2021-12-17,2024-09-30,"""SPOLU""","""STAN*ODS*KDU-ČSL*TOP09*Piráti""","""ANO2011*SPD""","""""","""vláda"""
"""Místopředsedkyně PSP Olga Richterová""","""2021ps_stenprot_104schuz_s104133.htm""","""/sqw/detail.sqw?id=6473""","""Já děkuji panu poslanci. S faktickou mám přihlášeného pana poslance Huberta Langa. Prosím, pane poslanče.""",2,"""ps""",2021,104,133,2024-05-29,"""PČR, PS 2021-...""","""104. schůze""",true,true,true,"""6473""",14,"""6473_9.html""","""PhDr. Olga Richterová, Ph.D.""","""Praha""","""f""",2021-10-09,2021-11-10,"""PirSTAN""","""Piráti""","""21. 1. 1985""","[{""člen"",""Organizační výbor"",""10. 11. 2021"",null}, {""místopředsed"",""Organizační výbor"",""10. 11. 2021"",null}, … {""člen"",""Skupina ČR - Spolková republika Německo"",""2. 3. 2022"",null}]",1985,1985-01-21,"""Olga Richterová""","[""PhDr."", ""Ph.D.""]","""vláda Petra Fialy s Piráty""",2021-12-17,2024-09-30,"""SPOLU""","""STAN*ODS*KDU-ČSL*TOP09*Piráti""","""ANO2011*SPD""","""""","""vláda"""
"""Místopředsedkyně PSP Věra Kovářová""","""2021ps_stenprot_053schuz_s053175.htm""","""/sqw/detail.sqw?id=6214""","""Děkuji, pane zpravodaji, a nyní se přihlásil s přednostním právem pan předseda Michálek a připraví se paní místopředsedkyně Klára Dostálová. Prosím, m…",2,"""ps""",2021,53,175,2023-03-24,"""PČR, PS 2021-...""","""53. schůze""",true,true,true,"""6214""",22,"""6214_9.html""","""Ing. Věra Kovářová""","""Středočeský""","""f""",2021-10-09,2021-11-10,"""PirSTAN""","""STAN""","""20. 1. 1964""","[{""člen"",""Organizační výbor"",""10. 11. 2021"",null}, {""místopředsed"",""Organizační výbor"",""10. 11. 2021"",null}, … {""člen"",""Skupina ČR - Paraguyaská republika"",""28. 3. 2024"",null}]",1964,1964-01-20,"""Věra Kovářová""","[""Ing.""]","""vláda Petra Fialy s Piráty""",2021-12-17,2024-09-30,"""SPOLU""","""STAN*ODS*KDU-ČSL*TOP09*Piráti""","""ANO2011*SPD""","""""","""vláda"""
"""Místopředsedkyně PSP Olga Richterová""","""2021ps_stenprot_079schuz_s079126.htm""","""/sqw/detail.sqw?id=6473""","""Já také děkuji pěkně za dodržení času. A my se nyní vracíme k řádně přihlášeným. Já ještě načtu... a to s přednostním právem... ano. Já ještě načtu om…",8,"""ps""",2021,79,126,2023-10-18,"""PČR, PS 2021-...""","""79. schůze""",true,true,true,"""6473""",68,"""6473_9.html""","""PhDr. Olga Richterová, Ph.D.""","""Praha""","""f""",2021-10-09,2021-11-10,"""PirSTAN""","""Piráti""","""21. 1. 1985""","[{""člen"",""Organizační výbor"",""10. 11. 2021"",null}, {""místopředsed"",""Organizační výbor"",""10. 11. 2021"",null}, … {""člen"",""Skupina ČR - Spolková republika Německo"",""2. 3. 2

In [1140]:
ministri = df.filter(pl.col("mluvci_id").str.contains("vlada.cz")).group_by_dynamic(index_column="datum",every="1d").agg(pl.col("pocet_slov").sum())

In [1141]:
alt.Chart(
    ministri.to_pandas()
).mark_line(
).encode(
    alt.X("datum:T"),
    alt.Y("pocet_slov:Q")
)

alt.Chart(...)

## Muži x ženy

In [1143]:
df.group_by(
    ["obdobi","gender"]
).agg(
    pl.col('pocet_slov').sum()
).pivot(
    index='obdobi',on='gender'
).with_columns(
    (pl.col('f') / ((pl.col('f')) + pl.col('m'))).alias('podil_zen')
).sort(
    by='obdobi'
)

obdobi,m,null,f,podil_zen
i32,u32,u32,u32,f64
1993,null,3623355,null,null
1996,null,3686739,null,null
1998,4303,6680611,728,0.144703
2002,5457437,394300,1265763,0.188268
2006,4347412,342482,902723,0.171943
2010,3423078,771450,684951,0.166735
2013,5959939,1859972,1065260,0.151634
2017,6762057,1928554,1223490,0.153213
2021,5195665,1619897,2598989,0.333432


In [1144]:
df.with_columns(
    pl.when(
        pl.col('mluvci').str.contains(r'ová[^\w]') | pl.col('mluvci').str.ends_with('á') | pl.col('mluvci').str.contains('oslankyně') | pl.col("mluvci").str.contains("sedkyně ")
    ).then(pl.lit('f')).otherwise(pl.lit('m')).alias('gender_naivni')
).group_by(
    ["obdobi","gender_naivni"]
).agg(
    pl.col('pocet_slov').sum()
).pivot(
    index='obdobi',on='gender_naivni'
).with_columns(
    (pl.col('f') / ((pl.col('f')) + pl.col('m'))).alias('podil_zen')
).sort(
    by='obdobi'
)

obdobi,m,f,podil_zen
i32,u32,u32,f64
1993,3421177,202178,0.055799
1996,3275035,411704,0.111672
1998,5683219,1002423,0.149937
2002,5818427,1299073,0.182518
2006,4646275,946342,0.169213
2010,3688584,1190895,0.244062
2013,7641783,1243388,0.13994
2017,8217237,1696864,0.171157
2021,6414119,3000432,0.318702


In [1145]:
df.filter(
    (pl.col('mluvci').str.contains(r'ová[^\w]') | pl.col('mluvci').str.ends_with('á') | pl.col('mluvci').str.contains('oslankyně') | pl.col("mluvci").str.contains("sedkyně "))
).group_by("mluvci").agg(pl.col("text").unique().len()).sort(by="text",descending=True)

mluvci,text
str,u32
"""Místopředsedkyně PSP Petra Buzková""",12131
"""Místopředsedkyně PSP Miroslava Němcová""",8824
"""Předsedkyně PSP Miroslava Němcová""",7610
"""Místopředsedkyně PSP Jitka Kupčová""",5795
"""Místopředsedkyně PSP Olga Richterová""",5601
"""Místopředsedkyně PSP Věra Kovářová""",5346
"""Místopředsedkyně PSP Jaroslava Jermanová""",4621
"""Předsedkyně PSP Markéta Pekarová Adamová""",4564
"""Místopředsedkyně PSP Lucie Talmanová""",3942


In [1146]:
df.filter(
    ~(pl.col('mluvci').str.contains(r'ová[^\w]') | pl.col('mluvci').str.ends_with('á') | pl.col('mluvci').str.contains('oslankyně') | pl.col("mluvci").str.contains("sedkyně "))
).group_by("mluvci").agg(pl.col("text").unique().len()).sort(by="text",descending=True)

mluvci,text
str,u32
"""Místopředseda PSP Vojtěch Filip""",27176
"""Místopředseda PSP Jan Kasal""",15685
"""Místopředseda PSP Ivan Langer""",14311
"""Místopředseda PSP Jan Bartošek""",13644
"""Místopředseda PSP Lubomír Zaorálek""",9702
"""Místopředseda PSP Petr Gazdík""",8925
"""Předseda PSP Lubomír Zaorálek""",7861
"""Místopředseda PSP Tomio Okamura""",7423
"""Místopředseda PSP Vojtěch Pikal""",7224


In [1147]:
df.filter(pl.col("gender") == "f").filter(
    ~(pl.col('mluvci').str.contains(r'ová[^\w]') | pl.col('mluvci').str.ends_with('á') | pl.col('mluvci').str.contains('oslankyně') | pl.col("mluvci").str.contains("sedkyně "))
).group_by("mluvci").agg(pl.col("text").unique().len()).sort(by="text",descending=True)

mluvci,text
str,u32
"""Náměstek primátora hl. m. Prahy Rudolf Blažek""",18
"""Guvernér ČNB Zdeněk Tůma""",12
"""Senátor Jaroslav Kubera""",10
"""Senátor Martin Mejstřík""",6
"""Senátor Josef Novotný""",5
"""Senátor Luděk Sefzig""",5
"""Hejtman Ústeckého kraje Jiří Šulc""",5
"""Pan Rudolf Blažek""",4
"""Senátor Adolf Jílek""",4


In [1148]:
df.filter(pl.col('mluvci') == 'Náměstek primátora hl. m. Prahy Rudolf Blažek')

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov,soubor_right,jmeno,kraj,gender,obdobi_od,obdobi_do,kandidatka,navrhovatel,narozeni_retezec,clenstvi,narozeni_rok,narozeni_datum,jmeno_ciste,tituly
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32,str,str,str,str,date,date,str,str,str,list[struct[4]],i64,date,str,list[str]
"""Náměstek primátora hl. m. Prahy Rudolf Blažek""","""2002ps_stenprot_016schuz_s016023.htm""","""/sqw/detail.sqw?id=64""","""Děkuji, hezký dobrý den. Vážený pane předsedající, vážené dámy, vážení pánové, podruhé před vás předstupuji s návrhem hlavního města Prahy na schválen…",3,"""ps""",2002,16,23,2003-05-13,"""Parlament České republiky, Poslanecká sněmovna 2002 – 2006""","""16. schůze""",true,true,true,"""64""",191,"""64_4.html""","""Ing. Jan Kasal""","""Vysočina""","""m""",2002-06-15,2006-06-15,"""K""","""KDU-ČSL""","""1951""","[{""člen"",""Organizační výbor"",""16. 7. 2002"",""15. 6. 2006""}, {""místopředsed"",""Organizační výbor"",""16. 7. 2002"",""15. 6. 2006""}, … {""předsed"",""Skupina ČR - Rusko a Eurasie"",""6. 3. 2003"",""10. 2. 2005""}]",1951,null,"""Jan Kasal""","[""Ing.""]"
"""Náměstek primátora hl. m. Prahy Rudolf Blažek""","""2002ps_stenprot_022schuz_s022009.htm""","""/sqw/detail.sqw?id=252""","""Vážený pane předsedo, vážené paní poslankyně, vážení páni poslanci, děkuji za udělení slova. K našemu návrhu bych chtěl uvést, že dnes před vás již po…",7,"""ps""",2002,22,9,2003-10-14,"""Parlament České republiky, Poslanecká sněmovna 2002 – 2006""","""22. schůze""",true,true,true,"""252""",172,"""252_4.html""","""PhDr. Lubomír Zaorálek""","""Moravskoslezský""","""m""",2002-06-15,2006-06-15,"""ČSSD""",null,"""1956""","[{""člen"",""Organizační výbor"",""16. 7. 2002"",""15. 6. 2006""}, {""předsed"",""Organizační výbor"",""16. 7. 2002"",""15. 6. 2006""}, … {""člen"",""Poslanecký klub České strany sociálně demokratické"",""21. 6. 2002"",""15. 6. 2006""}]",1956,null,"""Lubomír Zaorálek""","[""PhDr.""]"
"""Náměstek primátora hl. m. Prahy Rudolf Blažek""","""2002ps_stenprot_030schuz_s030161.htm""","""/sqw/detail.sqw?id=401""","""Vážená paní místopředsedkyně, vážené paní poslankyně, vážení poslanci, předkládám vám k posouzení návrh hl. m. Prahy na vydání zákona, kterým se mění …",7,"""ps""",2002,30,161,2004-03-30,"""Parlament České republiky, Poslanecká sněmovna 2002 – 2006""","""30. schůze""",true,true,true,"""401""",761,"""401_4.html""","""Miroslava Němcová""","""Vysočina""","""f""",2002-06-15,2006-06-15,"""ODS""",null,"""1952""","[{""člen"",""Organizační výbor"",""16. 7. 2002"",""15. 6. 2006""}, {""místopředsed"",""Organizační výbor"",""16. 7. 2002"",""15. 6. 2006""}, … {""člen"",""Skupina ČR - Izrael"",""11. 12. 2002"",""15. 6. 2006""}]",1952,null,"""Miroslava Němcová""",[]
"""Náměstek primátora hl. m. Prahy Rudolf Blažek""","""2002ps_stenprot_030schuz_s030163.htm""","""/sqw/detail.sqw?id=401""","""Děkuji, vážená paní předsedající. Dámy a pánové, v zastoupení hlavního města Prahy předkládám dnes druhý návrh zákona, a to návrh zákona, kterým se mě…",2,"""ps""",2002,30,163,2004-03-30,"""Parlament České republiky, Poslanecká sněmovna 2002 – 2006""","""30. schůze""",true,true,true,"""401""",1212,"""401_4.html""","""Miroslava Němcová""","""Vysočina""","""f""",2002-06-15,2006-06-15,"""ODS""",null,"""1952""","[{""člen"",""Organizační výbor"",""16. 7. 2002"",""15. 6. 2006""}, {""místopředsed"",""Organizační výbor"",""16. 7. 2002"",""15. 6. 2006""}, … {""člen"",""Skupina ČR - Izrael"",""11. 12. 2002"",""15. 6. 2006""}]",1952,null,"""Miroslava Němcová""",[]
"""Náměstek primátora hl. m. Prahy Rudolf Blažek""","""2002ps_stenprot_030schuz_s030165.htm""","""/sqw/detail.sqw?id=401""","""Vážená paní předsedající, vážené dámy, vážení pánové, jsem trošku rozpačitý, protože samozřejmě je to především problém velkých měst. O tom není disku…",7,"""ps""",2002,30,165,2004-03-30,"""Parlament České republiky, Po